# Binary variables on both production and consumption

## Penalty in the code

In [ ]:
import pandas as pd
import os
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from datetime import datetime

# Paths to directories
Data_path_GDrive = "G:/Il mio Drive/Thesis_Large_Files/Working Table"
Data_path_Production = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Production"
Data_path_Consumption = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption"
Data_path_Users = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Users"
results_dir = 'C:/Users/Nik/Documents/GitHub/Thesis/Results/Results_Binary/Final Results'
results_dir = os.path.join(results_dir, datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + 'BinaryProductionandConsumption')
os.makedirs(results_dir, exist_ok=True)

# Load Working Tables for every optimization try
WorkingTables_Path = 'C:/Users/Nik/Documents/GitHub/Thesis/CSV/Working Tables'
DF_m2_Residential = pd.read_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\ResidentialConsumption-4000\Modified_DF_m2_Residential.csv', sep=",", on_bad_lines='skip', header=0)
DF_m2_Industrial = pd.read_csv(os.path.join(WorkingTables_Path, 'DF_m2_Industrial.csv'), sep=",", on_bad_lines='skip', header=0)
DF_m2_Commercial = pd.read_csv(os.path.join(WorkingTables_Path, 'DF_m2_Commercial.csv'), sep=",", on_bad_lines='skip', header=0)

Residential_user_production_df = pd.read_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\ResidentialConsumption-4000\Modified_Residential_Production.csv', sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])
Industrial_user_production_df = pd.read_csv(os.path.join(WorkingTables_Path, 'Industrial_user_production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])
Commercial_user_production_df = pd.read_csv(os.path.join(WorkingTables_Path, 'Commercial_user_production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])

# Load New Consumption Curves
Residential_Consumption = pd.read_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\ResidentialConsumption-4000\Modified_Residential_Consumption.csv', index_col=0, parse_dates=True)
Industrial_Consumption = pd.read_csv('C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption/Industrial_Consumption.csv', index_col=0, parse_dates=True)
Commercial_Consumption = pd.read_csv('C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption/Commercial_Consumption.csv', index_col=0, parse_dates=True)

DF_Production = pd.read_csv(os.path.join(Data_path_Production, '2019_Production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])
print("ok")
# Ensure indices are in datetime format for alignment
Residential_Consumption.index = pd.to_datetime(Residential_Consumption.index)
Industrial_Consumption.index = pd.to_datetime(Industrial_Consumption.index)
Commercial_Consumption.index = pd.to_datetime(Commercial_Consumption.index)
DF_Production.index = pd.to_datetime(DF_Production.index)

# Use 'full_id' as the primary identifier
residential_users = DF_m2_Residential['full_id'].astype(str).tolist()
industrial_users = DF_m2_Industrial['full_id'].astype(str).tolist()
commercial_users = DF_m2_Commercial['full_id'].astype(str).tolist()

# Precompute area mappings to avoid repeated DataFrame lookups
residential_area_dict = DF_m2_Residential.set_index('full_id')['Area'].to_dict()
industrial_area_dict = DF_m2_Industrial.set_index('full_id')['Area'].to_dict()
commercial_area_dict = DF_m2_Commercial.set_index('full_id')['Area'].to_dict()

# Define the yearlist based on the number of hours
yearlist = pd.to_datetime(DF_Production.index)

# Dynamic consumption area constraints (e.g., set as 60% of the total available area)
Fixed_Res_Consumption_Area = 0.6 * DF_m2_Residential['Area'].sum()
Fixed_Ind_Consumption_Area = 0.6 * DF_m2_Industrial['Area'].sum()
Fixed_Com_Consumption_Area = 0.6 * DF_m2_Commercial['Area'].sum()
Total_Fixed_Consumption_Area = Fixed_Res_Consumption_Area + Fixed_Ind_Consumption_Area + Fixed_Com_Consumption_Area

# Create a new Gurobi model
m = gp.Model("Optimization_Model_ConsumptionProduction")

# Production and Consumption Binary Variables [0;1]
binary_vars_production_residential = m.addVars(residential_users, vtype=GRB.BINARY, name="binary_production_residential")
binary_vars_consumption_residential = m.addVars(residential_users, vtype=GRB.BINARY, name="binary_consumption_residential")

binary_vars_production_industrial = m.addVars(industrial_users, vtype=GRB.BINARY, name="binary_production_industrial")
binary_vars_consumption_industrial = m.addVars(industrial_users, vtype=GRB.BINARY, name="binary_consumption_industrial")

binary_vars_production_commercial = m.addVars(commercial_users, vtype=GRB.BINARY, name="binary_production_commercial")
binary_vars_consumption_commercial = m.addVars(commercial_users, vtype=GRB.BINARY, name="binary_consumption_commercial")

# Auxiliary variable for absolute deviation
deviation = m.addVars(yearlist, lb=0, vtype=GRB.CONTINUOUS, name="deviation")

# Penalty coefficient for not selecting users
penalty_coefficient = 5  # Adjusted to be less aggressive

# Pre-compute production terms for each user and hour
residential_prod_terms = {
    t: {user: Residential_user_production_df.loc[t, user] for user in residential_users if user in Residential_user_production_df.columns}
    for t in yearlist
}
industrial_prod_terms = {
    t: {user: Industrial_user_production_df.loc[t, user] for user in industrial_users if user in Industrial_user_production_df.columns}
    for t in yearlist
}
commercial_prod_terms = {
    t: {user: Commercial_user_production_df.loc[t, user] for user in commercial_users if user in Commercial_user_production_df.columns}
    for t in yearlist
}

# Pre-compute consumption terms for each user and hour
residential_cons_terms = {
    t: {user: Residential_Consumption[user].loc[t] * residential_area_dict[user]
        if t in Residential_Consumption.index else 0  # Check if the index exists
        for user in residential_users}
    for t in yearlist
}

industrial_cons_terms = {
    t: {user: Industrial_Consumption[user].loc[t] * industrial_area_dict[user]
        if t in Industrial_Consumption.index else 0  # Check if the index exists
        for user in industrial_users}
    for t in yearlist
}

commercial_cons_terms = {
    t: {user: Commercial_Consumption[user].loc[t] * commercial_area_dict[user]
        if t in Commercial_Consumption.index else 0  # Check if the index exists
        for user in commercial_users}
    for t in yearlist
}

# Calculate the difference between production and consumption for each hour using precomputed terms
for t in yearlist:
    production_res = gp.quicksum(binary_vars_production_residential[user] * residential_prod_terms[t].get(user, 0) for user in residential_users)
    consumption_res = gp.quicksum(binary_vars_consumption_residential[user] * residential_cons_terms[t].get(user, 0) for user in residential_users)
    
    production_ind = gp.quicksum(binary_vars_production_industrial[user] * industrial_prod_terms[t].get(user, 0) for user in industrial_users)
    consumption_ind = gp.quicksum(binary_vars_consumption_industrial[user] * industrial_cons_terms[t].get(user, 0) for user in industrial_users)
    
    production_com = gp.quicksum(binary_vars_production_commercial[user] * commercial_prod_terms[t].get(user, 0) for user in commercial_users)
    consumption_com = gp.quicksum(binary_vars_consumption_commercial[user] * commercial_cons_terms[t].get(user, 0) for user in commercial_users)
    
    difference = production_res - consumption_res + production_ind - consumption_ind + production_com - consumption_com
    
    # Add constraints for deviation
    m.addConstr(deviation[t] >= difference, name=f"PosDeviation_{t}")
    m.addConstr(deviation[t] >= -difference, name=f"NegDeviation_{t}")

# Constraint: Total selected consumption area must equal or exceed the fixed value
m.addConstr(
    gp.quicksum(binary_vars_consumption_residential[user] * residential_area_dict[user] for user in residential_users) + 
    gp.quicksum(binary_vars_consumption_industrial[user] * industrial_area_dict[user] for user in industrial_users) +
    gp.quicksum(binary_vars_consumption_commercial[user] * commercial_area_dict[user] for user in commercial_users) >= Total_Fixed_Consumption_Area,
    name="FixedConsumptionArea"
)

# Determine the peak production and consumption values for each user type
peak_production_residential = gp.max_(gp.quicksum(binary_vars_production_residential[user] * residential_prod_terms[t].get(user, 0) for user in residential_users) for t in yearlist)
peak_consumption_residential = gp.max_(gp.quicksum(binary_vars_consumption_residential[user] * residential_cons_terms[t].get(user, 0) for user in residential_users) for t in yearlist)

peak_production_industrial = gp.max_(gp.quicksum(binary_vars_production_industrial[user] * industrial_prod_terms[t].get(user, 0) for user in industrial_users) for t in yearlist)
peak_consumption_industrial = gp.max_(gp.quicksum(binary_vars_consumption_industrial[user] * industrial_cons_terms[t].get(user, 0) for user in industrial_users) for t in yearlist)

peak_production_commercial = gp.max_(gp.quicksum(binary_vars_production_commercial[user] * commercial_prod_terms[t].get(user, 0) for user in commercial_users) for t in yearlist)
peak_consumption_commercial = gp.max_(gp.quicksum(binary_vars_consumption_commercial[user] * commercial_cons_terms[t].get(user, 0) for user in commercial_users) for t in yearlist)

# Define a tolerance range for the order of magnitude constraint (e.g., within a factor of 10)
tolerance_factor = 10

# Add constraints to ensure peak production and peak consumption are within the same order of magnitude
m.addConstr(peak_production_residential <= tolerance_factor * peak_consumption_residential, name="PeakProdConsOrderResidential")
m.addConstr(peak_consumption_residential <= tolerance_factor * peak_production_residential, name="PeakConsProdOrderResidential")

m.addConstr(peak_production_industrial <= tolerance_factor * peak_consumption_industrial, name="PeakProdConsOrderIndustrial")
m.addConstr(peak_consumption_industrial <= tolerance_factor * peak_production_industrial, name="PeakConsProdOrderIndustrial")

m.addConstr(peak_production_commercial <= tolerance_factor * peak_consumption_commercial, name="PeakProdConsOrderCommercial")
m.addConstr(peak_consumption_commercial <= tolerance_factor * peak_production_commercial, name="PeakConsProdOrderCommercial")

# Add diversification constraints
diversification_factor = 0.8  # Allow some flexibility, e.g., up to 80% imbalance allowed

# Residential diversification constraint
m.addConstr(
    gp.quicksum(binary_vars_production_residential[user] for user in residential_users) >= diversification_factor * gp.quicksum(binary_vars_consumption_residential[user] for user in residential_users),
    name="ResDiversificationProductionConsumption"
)
m.addConstr(
    gp.quicksum(binary_vars_consumption_residential[user] for user in residential_users) >= diversification_factor * gp.quicksum(binary_vars_production_residential[user] for user in residential_users),
    name="ResDiversificationConsumptionProduction"
)

# Industrial diversification constraint
m.addConstr(
    gp.quicksum(binary_vars_production_industrial[user] for user in industrial_users) >= diversification_factor * gp.quicksum(binary_vars_consumption_industrial[user] for user in industrial_users),
    name="IndDiversificationProductionConsumption"
)
m.addConstr(
    gp.quicksum(binary_vars_consumption_industrial[user] for user in industrial_users) >= diversification_factor * gp.quicksum(binary_vars_production_industrial[user] for user in industrial_users),
    name="IndDiversificationConsumptionProduction"
)

# Commercial diversification constraint
m.addConstr(
    gp.quicksum(binary_vars_production_commercial[user] for user in commercial_users) >= diversification_factor * gp.quicksum(binary_vars_consumption_commercial[user] for user in commercial_users),
    name="ComDiversificationProductionConsumption"
)
m.addConstr(
    gp.quicksum(binary_vars_consumption_commercial[user] for user in commercial_users) >= diversification_factor * gp.quicksum(binary_vars_production_commercial[user] for user in commercial_users),
    name="ComDiversificationConsumptionProduction"
)

# Objective function: minimize the deviation and penalize unselected users
penalty = (
    penalty_coefficient * (
        gp.quicksum(1 - binary_vars_production_residential[user] for user in residential_users) +
        gp.quicksum(1 - binary_vars_consumption_residential[user] for user in residential_users) +
        gp.quicksum(1 - binary_vars_production_industrial[user] for user in industrial_users) +
        gp.quicksum(1 - binary_vars_consumption_industrial[user] for user in industrial_users) +
        gp.quicksum(1 - binary_vars_production_commercial[user] for user in commercial_users) +
        gp.quicksum(1 - binary_vars_consumption_commercial[user] for user in commercial_users)
    )
)

# Minimize deviation and add penalization
m.setObjective(gp.quicksum(deviation[t] for t in yearlist)+ penalty, GRB.MINIMIZE)

# Set solver parameters to potentially reduce runtime and monitor output
m.Params.OutputFlag = 1  # Ensure detailed output
m.Params.MIPGap = 0.01  # Set gap tolerance to 1% to allow faster convergence
m.Params.TimeLimit = 10800  # Set a time limit of 1.5 hours (5400 seconds) to prevent excessive runtimes
m.Params.Presolve = 2  # Aggressive presolve level
m.Params.Cuts = 2  # Apply aggressive cuts
m.Params.Heuristics = 0.5  # Increase the emphasis on heuristics to find feasible solutions faster

# Enable logging to file for detailed analysis later
m.Params.LogFile = os.path.join(results_dir, "optimization_log.txt")

# Optimize the model
m.optimize()

# Check for solution status
if m.Status == GRB.OPTIMAL:
    print("Optimal solution found!")
elif m.Status == GRB.INFEASIBLE:
    print("Model is infeasible.")
    m.computeIIS()  # Compute Irreducible Inconsistent Subsystem (IIS) to understand infeasibility
    m.write(os.path.join(results_dir, "model.ilp"))  # Save IIS to a file
elif m.Status == GRB.TIME_LIMIT:
    print("Optimization stopped due to time limit.")
else:
    print(f"Optimization ended with status {m.Status}.")

# Display the results
for v in m.getVars():
    print(f'{v.varName}: {v.x}')

print(f'Objective: {m.objVal}')

# Extract results for each user type
RP_values = {user: binary_vars_production_residential[user].X for user in residential_users}
CP_values = {user: binary_vars_consumption_residential[user].X for user in residential_users}
IP_values = {user: binary_vars_production_industrial[user].X for user in industrial_users}
CIP_values = {user: binary_vars_consumption_industrial[user].X for user in industrial_users}
CP_values_com = {user: binary_vars_production_commercial[user].X for user in commercial_users}
CIP_values_com = {user: binary_vars_consumption_commercial[user].X for user in commercial_users}

# Create new DataFrames with the chosen binary variables
DF_m2_Residential['Chosen_Production'] = [RP_values[user] for user in DF_m2_Residential['full_id']]
DF_m2_Residential['Chosen_Consumption'] = [CP_values[user] for user in DF_m2_Residential['full_id']]
DF_m2_Industrial['Chosen_Production'] = [IP_values[user] for user in DF_m2_Industrial['full_id']]
DF_m2_Industrial['Chosen_Consumption'] = [CIP_values[user] for user in DF_m2_Industrial['full_id']]
DF_m2_Commercial['Chosen_Production'] = [CP_values_com[user] for user in DF_m2_Commercial['full_id']]
DF_m2_Commercial['Chosen_Consumption'] = [CIP_values_com[user] for user in DF_m2_Commercial['full_id']]

# Save the results to CSV files
DF_m2_Residential.to_csv(os.path.join(results_dir, 'New_residential_DF.csv'))
DF_m2_Industrial.to_csv(os.path.join(results_dir, 'New_Industrial_DF.csv'))
DF_m2_Commercial.to_csv(os.path.join(results_dir, 'New_commercial_DF.csv'))

# Calculate the number of chosen users versus the total number of users for each category
num_residential_chosen_production = len(DF_m2_Residential[DF_m2_Residential['Chosen_Production'] == 1])
num_residential_total = len(DF_m2_Residential)
num_residential_chosen_consumption = len(DF_m2_Residential[DF_m2_Residential['Chosen_Consumption'] == 1])

num_industrial_chosen_production = len(DF_m2_Industrial[DF_m2_Industrial['Chosen_Production'] == 1])
num_industrial_total = len(DF_m2_Industrial)
num_industrial_chosen_consumption = len(DF_m2_Industrial[DF_m2_Industrial['Chosen_Consumption'] == 1])

num_commercial_chosen_production = len(DF_m2_Commercial[DF_m2_Commercial['Chosen_Production'] == 1])
num_commercial_total = len(DF_m2_Commercial)
num_commercial_chosen_consumption = len(DF_m2_Commercial[DF_m2_Commercial['Chosen_Consumption'] == 1])

# Print the amounts
print(f'Residential Production: Chosen = {num_residential_chosen_production}, Total = {num_residential_total}')
print(f'Residential Consumption: Chosen = {num_residential_chosen_consumption}, Total = {num_residential_total}')
print(f'Industrial Production: Chosen = {num_industrial_chosen_production}, Total = {num_industrial_total}')
print(f'Industrial Consumption: Chosen = {num_industrial_chosen_consumption}, Total = {num_industrial_total}')
print(f'Commercial Production: Chosen = {num_commercial_chosen_production}, Total = {num_commercial_total}')
print(f'Commercial Consumption: Chosen = {num_commercial_chosen_consumption}, Total = {num_commercial_total}')

# Summary of Results
print("\nSummary of Optimization Results:")
print(f"Total Residential Production Users Chosen: {num_residential_chosen_production} out of {num_residential_total} ({(num_residential_chosen_production / num_residential_total) * 100:.2f}%)")
print(f"Total Residential Consumption Users Chosen: {num_residential_chosen_consumption} out of {num_residential_total} ({(num_residential_chosen_consumption / num_residential_total) * 100:.2f}%)")

print(f"Total Industrial Production Users Chosen: {num_industrial_chosen_production} out of {num_industrial_total} ({(num_industrial_chosen_production / num_industrial_total) * 100:.2f}%)")
print(f"Total Industrial Consumption Users Chosen: {num_industrial_chosen_consumption} out of {num_industrial_total} ({(num_industrial_chosen_consumption / num_industrial_total) * 100:.2f}%)")

print(f"Total Commercial Production Users Chosen: {num_commercial_chosen_production} out of {num_commercial_total} ({(num_commercial_chosen_production / num_commercial_total) * 100:.2f}%)")
print(f"Total Commercial Consumption Users Chosen: {num_commercial_chosen_consumption} out of {num_commercial_total} ({(num_commercial_chosen_consumption / num_commercial_total) * 100:.2f}%)")

# Check if production and consumption constraints are met
residential_prod_area = DF_m2_Residential[DF_m2_Residential['Chosen_Production'] == 1]['Area'].sum()
residential_cons_area = DF_m2_Residential[DF_m2_Residential['Chosen_Consumption'] == 1]['Area'].sum()

industrial_prod_area = DF_m2_Industrial[DF_m2_Industrial['Chosen_Production'] == 1]['Area'].sum()
industrial_cons_area = DF_m2_Industrial[DF_m2_Industrial['Chosen_Consumption'] == 1]['Area'].sum()

commercial_prod_area = DF_m2_Commercial[DF_m2_Commercial['Chosen_Production'] == 1]['Area'].sum()
commercial_cons_area = DF_m2_Commercial[DF_m2_Commercial['Chosen_Consumption'] == 1]['Area'].sum()

print(f"\nResidential Production Area: {residential_prod_area:.2f} | Consumption Area: {residential_cons_area:.2f} (Required >= {Fixed_Res_Consumption_Area:.2f})")
print(f"Industrial Production Area: {industrial_prod_area:.2f} | Consumption Area: {industrial_cons_area:.2f} (Required >= {Fixed_Ind_Consumption_Area:.2f})")
print(f"Commercial Production Area: {commercial_prod_area:.2f} | Consumption Area: {commercial_cons_area:.2f} (Required >= {Fixed_Com_Consumption_Area:.2f})")

# Check if diversification constraints are met
print("\nDiversification Checks:")
residential_prod_count = len(DF_m2_Residential[DF_m2_Residential['Chosen_Production'] == 1])
residential_cons_count = len(DF_m2_Residential[DF_m2_Residential['Chosen_Consumption'] == 1])

industrial_prod_count = len(DF_m2_Industrial[DF_m2_Industrial['Chosen_Production'] == 1])
industrial_cons_count = len(DF_m2_Industrial[DF_m2_Industrial['Chosen_Consumption'] == 1])

commercial_prod_count = len(DF_m2_Commercial[DF_m2_Commercial['Chosen_Production'] == 1])
commercial_cons_count = len(DF_m2_Commercial[DF_m2_Commercial['Chosen_Consumption'] == 1])

print(f"Residential Production Users Chosen: {residential_prod_count} | Consumption Users Chosen: {residential_cons_count}")
print(f"Industrial Production Users Chosen: {industrial_prod_count} | Consumption Users Chosen: {industrial_cons_count}")
print(f"Commercial Production Users Chosen: {commercial_prod_count} | Consumption Users Chosen: {commercial_cons_count}")

# Ensure the diversification factor is respected
if residential_prod_count >= diversification_factor * residential_cons_count and residential_cons_count >= diversification_factor * residential_prod_count:
    print("Residential diversification constraint is satisfied.")
else:
    print("Residential diversification constraint is NOT satisfied.")

if industrial_prod_count >= diversification_factor * industrial_cons_count and industrial_cons_count >= diversification_factor * industrial_prod_count:
    print("Industrial diversification constraint is satisfied.")
else:
    print("Industrial diversification constraint is NOT satisfied.")

if commercial_prod_count >= diversification_factor * commercial_cons_count and commercial_cons_count >= diversification_factor * commercial_prod_count:
    print("Commercial diversification constraint is satisfied.")
else:
    print("Commercial diversification constraint is NOT satisfied.")

above problem with the peak consumption as sum

added peak consumption as variable

In [3]:
import pandas as pd
import os
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from datetime import datetime

# Paths to directories
Data_path_GDrive = "G:/Il mio Drive/Thesis_Large_Files/Working Table"
Data_path_Production = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Production"
Data_path_Consumption = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption"
Data_path_Users = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Users"
results_dir = 'C:/Users/Nik/Documents/GitHub/Thesis/Results/Results_Binary/Final Results'
results_dir = os.path.join(results_dir, datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + 'BinaryProductionandConsumption')
os.makedirs(results_dir, exist_ok=True)

# Load Working Tables for every optimization try
WorkingTables_Path = 'C:/Users/Nik/Documents/GitHub/Thesis/CSV/Working Tables'
DF_m2_Residential = pd.read_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\ResidentialConsumption-4000\Modified_DF_m2_Residential.csv', sep=",", on_bad_lines='skip', header=0)
DF_m2_Industrial = pd.read_csv(os.path.join(WorkingTables_Path, 'DF_m2_Industrial.csv'), sep=",", on_bad_lines='skip', header=0)
DF_m2_Commercial = pd.read_csv(os.path.join(WorkingTables_Path, 'DF_m2_Commercial.csv'), sep=",", on_bad_lines='skip', header=0)

Residential_user_production_df = pd.read_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\ResidentialConsumption-4000\Modified_Residential_Production.csv', sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])
Industrial_user_production_df = pd.read_csv(os.path.join(WorkingTables_Path, 'Industrial_user_production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])
Commercial_user_production_df = pd.read_csv(os.path.join(WorkingTables_Path, 'Commercial_user_production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])

# Load New Consumption Curves
Residential_Consumption = pd.read_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\ResidentialConsumption-4000\Modified_Residential_Consumption.csv', index_col=0, parse_dates=True)
Industrial_Consumption = pd.read_csv('C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption/Industrial_Consumption.csv', index_col=0, parse_dates=True)
Commercial_Consumption = pd.read_csv('C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption/Commercial_Consumption.csv', index_col=0, parse_dates=True)

DF_Production = pd.read_csv(os.path.join(Data_path_Production, '2019_Production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])
print("ok")

# Ensure indices are in datetime format for alignment
Residential_Consumption.index = pd.to_datetime(Residential_Consumption.index)
Industrial_Consumption.index = pd.to_datetime(Industrial_Consumption.index)
Commercial_Consumption.index = pd.to_datetime(Commercial_Consumption.index)
DF_Production.index = pd.to_datetime(DF_Production.index)

# Use 'full_id' as the primary identifier
residential_users = DF_m2_Residential['full_id'].astype(str).tolist()
industrial_users = DF_m2_Industrial['full_id'].astype(str).tolist()
commercial_users = DF_m2_Commercial['full_id'].astype(str).tolist()

# Precompute area mappings to avoid repeated DataFrame lookups
residential_area_dict = DF_m2_Residential.set_index('full_id')['Area'].to_dict()
industrial_area_dict = DF_m2_Industrial.set_index('full_id')['Area'].to_dict()
commercial_area_dict = DF_m2_Commercial.set_index('full_id')['Area'].to_dict()

# Define the yearlist based on the number of hours
yearlist = pd.to_datetime(DF_Production.index)

# Dynamic consumption area constraints (e.g., set as 60% of the total available area)
Fixed_Res_Consumption_Area = 0.6 * DF_m2_Residential['Area'].sum()
Fixed_Ind_Consumption_Area = 0.6 * DF_m2_Industrial['Area'].sum()
Fixed_Com_Consumption_Area = 0.6 * DF_m2_Commercial['Area'].sum()
Total_Fixed_Consumption_Area = Fixed_Res_Consumption_Area + Fixed_Ind_Consumption_Area + Fixed_Com_Consumption_Area

# Create a new Gurobi model
m = gp.Model("Optimization_Model_ConsumptionProduction")

# Production and Consumption Binary Variables [0;1]
binary_vars_production_residential = m.addVars(residential_users, vtype=GRB.BINARY, name="binary_production_residential")
binary_vars_consumption_residential = m.addVars(residential_users, vtype=GRB.BINARY, name="binary_consumption_residential")

binary_vars_production_industrial = m.addVars(industrial_users, vtype=GRB.BINARY, name="binary_production_industrial")
binary_vars_consumption_industrial = m.addVars(industrial_users, vtype=GRB.BINARY, name="binary_consumption_industrial")

binary_vars_production_commercial = m.addVars(commercial_users, vtype=GRB.BINARY, name="binary_production_commercial")
binary_vars_consumption_commercial = m.addVars(commercial_users, vtype=GRB.BINARY, name="binary_consumption_commercial")

# Auxiliary variable for absolute deviation
deviation = m.addVars(yearlist, lb=0, vtype=GRB.CONTINUOUS, name="deviation")

# Penalty coefficient for not selecting users
penalty_coefficient = 5  # Adjusted to be less aggressive

# Pre-compute production terms for each user and hour
residential_prod_terms = {
    t: {user: Residential_user_production_df.loc[t, user] for user in residential_users if user in Residential_user_production_df.columns}
    for t in yearlist
}
industrial_prod_terms = {
    t: {user: Industrial_user_production_df.loc[t, user] for user in industrial_users if user in Industrial_user_production_df.columns}
    for t in yearlist
}
commercial_prod_terms = {
    t: {user: Commercial_user_production_df.loc[t, user] for user in commercial_users if user in Commercial_user_production_df.columns}
    for t in yearlist
}

# Pre-compute consumption terms for each user and hour
residential_cons_terms = {
    t: {user: Residential_Consumption[user].loc[t] * residential_area_dict[user]
        if t in Residential_Consumption.index else 0  # Check if the index exists
        for user in residential_users}
    for t in yearlist
}

industrial_cons_terms = {
    t: {user: Industrial_Consumption[user].loc[t] * industrial_area_dict[user]
        if t in Industrial_Consumption.index else 0  # Check if the index exists
        for user in industrial_users}
    for t in yearlist
}

commercial_cons_terms = {
    t: {user: Commercial_Consumption[user].loc[t] * commercial_area_dict[user]
        if t in Commercial_Consumption.index else 0  # Check if the index exists
        for user in commercial_users}
    for t in yearlist
}

# Calculate the difference between production and consumption for each hour using precomputed terms
for t in yearlist:
    production_res = gp.quicksum(binary_vars_production_residential[user] * residential_prod_terms[t].get(user, 0) for user in residential_users)
    consumption_res = gp.quicksum(binary_vars_consumption_residential[user] * residential_cons_terms[t].get(user, 0) for user in residential_users)
    
    production_ind = gp.quicksum(binary_vars_production_industrial[user] * industrial_prod_terms[t].get(user, 0) for user in industrial_users)
    consumption_ind = gp.quicksum(binary_vars_consumption_industrial[user] * industrial_cons_terms[t].get(user, 0) for user in industrial_users)
    
    production_com = gp.quicksum(binary_vars_production_commercial[user] * commercial_prod_terms[t].get(user, 0) for user in commercial_users)
    consumption_com = gp.quicksum(binary_vars_consumption_commercial[user] * commercial_cons_terms[t].get(user, 0) for user in commercial_users)
    
    difference = production_res - consumption_res + production_ind - consumption_ind + production_com - consumption_com
    
    # Add constraints for deviation
    m.addConstr(deviation[t] >= difference, name=f"PosDeviation_{t}")
    m.addConstr(deviation[t] >= -difference, name=f"NegDeviation_{t}")

# Constraint: Total selected consumption area must equal or exceed the fixed value
m.addConstr(
    gp.quicksum(binary_vars_consumption_residential[user] * residential_area_dict[user] for user in residential_users) + 
    gp.quicksum(binary_vars_consumption_industrial[user] * industrial_area_dict[user] for user in industrial_users) +
    gp.quicksum(binary_vars_consumption_commercial[user] * commercial_area_dict[user] for user in commercial_users) >= Total_Fixed_Consumption_Area,
    name="FixedConsumptionArea"
)

# Add diversification constraint
diversification_coefficient = 0.5  # Example value for diversification coefficient
m.addConstr(
    gp.quicksum(binary_vars_production_residential[user] for user in residential_users) +
    gp.quicksum(binary_vars_production_industrial[user] for user in industrial_users) +
    gp.quicksum(binary_vars_production_commercial[user] for user in commercial_users) >= 
    diversification_coefficient * (
        gp.quicksum(binary_vars_consumption_residential[user] for user in residential_users) +
        gp.quicksum(binary_vars_consumption_industrial[user] for user in industrial_users) +
        gp.quicksum(binary_vars_consumption_commercial[user] for user in commercial_users)
    ),
    name="DiversificationConstraint"
)

# Peak production and consumption constraints (same order of magnitude)
tolerance_factor = 1  # Adjust tolerance as needed

# Compute peak production and consumption
peak_production_residential = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="PeakProductionResidential")
peak_consumption_residential = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="PeakConsumptionResidential")
peak_production_industrial = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="PeakProductionIndustrial")
peak_consumption_industrial = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="PeakConsumptionIndustrial")
peak_production_commercial = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="PeakProductionCommercial")
peak_consumption_commercial = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="PeakConsumptionCommercial")

# Constraints for peak production and consumption per sector
for t in yearlist:
    m.addConstr(peak_production_residential >= gp.quicksum(binary_vars_production_residential[user] * residential_prod_terms[t].get(user, 0) for user in residential_users), name=f"PeakProdRes_{t}")
    m.addConstr(peak_consumption_residential >= gp.quicksum(binary_vars_consumption_residential[user] * residential_cons_terms[t].get(user, 0) for user in residential_users), name=f"PeakConsRes_{t}")
    
    m.addConstr(peak_production_industrial >= gp.quicksum(binary_vars_production_industrial[user] * industrial_prod_terms[t].get(user, 0) for user in industrial_users), name=f"PeakProdInd_{t}")
    m.addConstr(peak_consumption_industrial >= gp.quicksum(binary_vars_consumption_industrial[user] * industrial_cons_terms[t].get(user, 0) for user in industrial_users), name=f"PeakConsInd_{t}")
    
    m.addConstr(peak_production_commercial >= gp.quicksum(binary_vars_production_commercial[user] * commercial_prod_terms[t].get(user, 0) for user in commercial_users), name=f"PeakProdCom_{t}")
    m.addConstr(peak_consumption_commercial >= gp.quicksum(binary_vars_consumption_commercial[user] * commercial_cons_terms[t].get(user, 0) for user in commercial_users), name=f"PeakConsCom_{t}")

# Add constraints to ensure peak production and peak consumption are within the same order of magnitude
m.addConstr(peak_production_residential <= tolerance_factor * peak_consumption_residential, name="PeakProdConsOrderResidential")
m.addConstr(peak_consumption_residential <= tolerance_factor * peak_production_residential, name="PeakConsProdOrderResidential")

m.addConstr(peak_production_industrial <= tolerance_factor * peak_consumption_industrial, name="PeakProdConsOrderIndustrial")
m.addConstr(peak_consumption_industrial <= tolerance_factor * peak_production_industrial, name="PeakConsProdOrderIndustrial")

m.addConstr(peak_production_commercial <= tolerance_factor * peak_consumption_commercial, name="PeakProdConsOrderCommercial")
m.addConstr(peak_consumption_commercial <= tolerance_factor * peak_production_commercial, name="PeakConsProdOrderCommercial")

# Objective function: minimize the deviation and penalize unselected users
penalty = (
    penalty_coefficient * (
        gp.quicksum(1 - binary_vars_production_residential[user] for user in residential_users) +
        gp.quicksum(1 - binary_vars_consumption_residential[user] for user in residential_users) +
        gp.quicksum(1 - binary_vars_production_industrial[user] for user in industrial_users) +
        gp.quicksum(1 - binary_vars_consumption_industrial[user] for user in industrial_users) +
        gp.quicksum(1 - binary_vars_production_commercial[user] for user in commercial_users) +
        gp.quicksum(1 - binary_vars_consumption_commercial[user] for user in commercial_users)
    )
)

# Minimize deviation and add penalization
m.setObjective(gp.quicksum(deviation[t] for t in yearlist) + penalty, GRB.MINIMIZE)

# Set solver parameters to potentially reduce runtime and monitor output
m.Params.OutputFlag = 1  # Ensure detailed output
m.Params.MIPGap = 0.01  # Set gap tolerance to 1% to allow faster convergence
m.Params.TimeLimit = 10800  # Set a time limit of 3 hours
m.Params.Presolve = 2  # Aggressive presolve level
m.Params.Cuts = 2  # Apply aggressive cuts
m.Params.Heuristics = 0.5  # Increase the emphasis on heuristics to find feasible solutions faster

# Enable logging to file for detailed analysis later
m.Params.LogFile = os.path.join(results_dir, "optimization_log.txt")

# Optimize the model
m.optimize()

# Check for solution status
if m.Status == GRB.OPTIMAL:
    print("Optimal solution found!")
elif m.Status == GRB.INFEASIBLE:
    print("Model is infeasible.")
    m.computeIIS()  # Compute Irreducible Inconsistent Subsystem (IIS) to understand infeasibility
    m.write(os.path.join(results_dir, "model.ilp"))  # Save IIS to a file
elif m.Status == GRB.TIME_LIMIT:
    print("Optimization stopped due to time limit.")
else:
    print(f"Optimization ended with status {m.Status}.")

# Display the results
for v in m.getVars():
    print(f'{v.varName}: {v.x}')

print(f'Objective: {m.objVal}')

# Extract results for each user type
RP_values = {user: binary_vars_production_residential[user].X for user in residential_users}
CP_values = {user: binary_vars_consumption_residential[user].X for user in residential_users}
IP_values = {user: binary_vars_production_industrial[user].X for user in industrial_users}
CIP_values = {user: binary_vars_consumption_industrial[user].X for user in industrial_users}
CP_values_com = {user: binary_vars_production_commercial[user].X for user in commercial_users}
CIP_values_com = {user: binary_vars_consumption_commercial[user].X for user in commercial_users}

# Create new DataFrames with the chosen binary variables
DF_m2_Residential['Chosen_Production'] = [RP_values[user] for user in DF_m2_Residential['full_id']]
DF_m2_Residential['Chosen_Consumption'] = [CP_values[user] for user in DF_m2_Residential['full_id']]
DF_m2_Industrial['Chosen_Production'] = [IP_values[user] for user in DF_m2_Industrial['full_id']]
DF_m2_Industrial['Chosen_Consumption'] = [CIP_values[user] for user in DF_m2_Industrial['full_id']]
DF_m2_Commercial['Chosen_Production'] = [CP_values_com[user] for user in DF_m2_Commercial['full_id']]
DF_m2_Commercial['Chosen_Consumption'] = [CIP_values_com[user] for user in DF_m2_Commercial['full_id']]

# Save the results to CSV files
DF_m2_Residential.to_csv(os.path.join(results_dir, 'New_residential_DF.csv'))
DF_m2_Industrial.to_csv(os.path.join(results_dir, 'New_Industrial_DF.csv'))
DF_m2_Commercial.to_csv(os.path.join(results_dir, 'New_commercial_DF.csv'))

# Print the amounts
print(f'Residential Production: Chosen = {num_residential_chosen_production}, Total = {num_residential_total}')
print(f'Residential Consumption: Chosen = {num_residential_chosen_consumption}, Total = {num_residential_total}')
print(f'Industrial Production: Chosen = {num_industrial_chosen_production}, Total = {num_industrial_total}')
print(f'Industrial Consumption: Chosen = {num_industrial_chosen_consumption}, Total = {num_industrial_total}')
print(f'Commercial Production: Chosen = {num_commercial_chosen_production}, Total = {num_commercial_total}')
print(f'Commercial Consumption: Chosen = {num_commercial_chosen_consumption}, Total = {num_commercial_total}')

# Summary of Results
print("\nSummary of Optimization Results:")
print(f"Total Residential Production Users Chosen: {num_residential_chosen_production} out of {num_residential_total} ({(num_residential_chosen_production / num_residential_total) * 100:.2f}%)")
print(f"Total Residential Consumption Users Chosen: {num_residential_chosen_consumption} out of {num_residential_total} ({(num_residential_chosen_consumption / num_residential_total) * 100:.2f}%)")

print(f"Total Industrial Production Users Chosen: {num_industrial_chosen_production} out of {num_industrial_total} ({(num_industrial_chosen_production / num_industrial_total) * 100:.2f}%)")
print(f"Total Industrial Consumption Users Chosen: {num_industrial_chosen_consumption} out of {num_industrial_total} ({(num_industrial_chosen_consumption / num_industrial_total) * 100:.2f}%)")

print(f"Total Commercial Production Users Chosen: {num_commercial_chosen_production} out of {num_commercial_total} ({(num_commercial_chosen_production / num_commercial_total) * 100:.2f}%)")
print(f"Total Commercial Consumption Users Chosen: {num_commercial_chosen_consumption} out of {num_commercial_total} ({(num_commercial_chosen_consumption / num_commercial_total) * 100:.2f}%)")


ok
Set parameter MIPGap to value 0.01
Set parameter TimeLimit to value 10800
Set parameter Presolve to value 2
Set parameter Cuts to value 2
Set parameter Heuristics to value 0.5
Set parameter LogFile to value "C:/Users/Nik/Documents/GitHub/Thesis/Results/Results_Binary/Final Results\2024-10-13_17-58-48BinaryProductionandConsumption\optimization_log.txt"
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 70088 rows, 17090 columns and 162812616 nonzeros
Model fingerprint: 0x226a486d
Variable types: 8766 continuous, 8324 integer (8324 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+05]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+04, 9e+04]
Presolve removed 0 rows and 0 columns (presolve time = 35s) ...
Presolve re

NameError: name 'num_residential_chosen_production' is not defined

### diversification factor is on the total selected users

In [ ]:
import pandas as pd
import os
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from datetime import datetime

# Paths to directories
Data_path_GDrive = "G:/Il mio Drive/Thesis_Large_Files/Working Table"
Data_path_Production = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Production"
Data_path_Consumption = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption"
Data_path_Users = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Users"
results_dir = 'C:/Users/Nik/Documents/GitHub/Thesis/Results/Results_Binary/Final Results'
results_dir = os.path.join(results_dir, datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + 'BinaryProductionandConsumption')
os.makedirs(results_dir, exist_ok=True)

# Load Working Tables for every optimization try
WorkingTables_Path = 'C:/Users/Nik/Documents/GitHub/Thesis/CSV/Working Tables'
DF_m2_Residential = pd.read_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\ResidentialConsumption-4000\Modified_DF_m2_Residential.csv', sep=",", on_bad_lines='skip', header=0)
DF_m2_Industrial = pd.read_csv(os.path.join(WorkingTables_Path, 'DF_m2_Industrial.csv'), sep=",", on_bad_lines='skip', header=0)
DF_m2_Commercial = pd.read_csv(os.path.join(WorkingTables_Path, 'DF_m2_Commercial.csv'), sep=",", on_bad_lines='skip', header=0)

Residential_user_production_df = pd.read_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\ResidentialConsumption-4000\Modified_Residential_Production.csv', sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])
Industrial_user_production_df = pd.read_csv(os.path.join(WorkingTables_Path, 'Industrial_user_production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])
Commercial_user_production_df = pd.read_csv(os.path.join(WorkingTables_Path, 'Commercial_user_production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])

# Load New Consumption Curves
Residential_Consumption = pd.read_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\ResidentialConsumption-4000\Modified_Residential_Consumption.csv', index_col=0, parse_dates=True)
Industrial_Consumption = pd.read_csv('C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption/Industrial_Consumption.csv', index_col=0, parse_dates=True)
Commercial_Consumption = pd.read_csv('C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption/Commercial_Consumption.csv', index_col=0, parse_dates=True)

DF_Production = pd.read_csv(os.path.join(Data_path_Production, '2019_Production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])

# Ensure indices are in datetime format for alignment
Residential_Consumption.index = pd.to_datetime(Residential_Consumption.index)
Industrial_Consumption.index = pd.to_datetime(Industrial_Consumption.index)
Commercial_Consumption.index = pd.to_datetime(Commercial_Consumption.index)
DF_Production.index = pd.to_datetime(DF_Production.index)

# Use 'full_id' as the primary identifier
residential_users = DF_m2_Residential['full_id'].astype(str).tolist()
industrial_users = DF_m2_Industrial['full_id'].astype(str).tolist()
commercial_users = DF_m2_Commercial['full_id'].astype(str).tolist()

# Precompute area mappings to avoid repeated DataFrame lookups
residential_area_dict = DF_m2_Residential.set_index('full_id')['Area'].to_dict()
industrial_area_dict = DF_m2_Industrial.set_index('full_id')['Area'].to_dict()
commercial_area_dict = DF_m2_Commercial.set_index('full_id')['Area'].to_dict()

# Define the yearlist based on the number of hours
yearlist = pd.to_datetime(DF_Production.index)

# Dynamic consumption area constraints (e.g., set as 60% of the total available area)
Fixed_Res_Consumption_Area = 0.6 * DF_m2_Residential['Area'].sum()
Fixed_Ind_Consumption_Area = 0.6 * DF_m2_Industrial['Area'].sum()
Fixed_Com_Consumption_Area = 0.6 * DF_m2_Commercial['Area'].sum()
Total_Fixed_Consumption_Area = Fixed_Res_Consumption_Area + Fixed_Ind_Consumption_Area + Fixed_Com_Consumption_Area

# Create a new Gurobi model
m = gp.Model("Optimization_Model_ConsumptionProduction")

# Production and Consumption Binary Variables [0;1]
binary_vars_production_residential = m.addVars(residential_users, vtype=GRB.BINARY, name="binary_production_residential")
binary_vars_consumption_residential = m.addVars(residential_users, vtype=GRB.BINARY, name="binary_consumption_residential")

binary_vars_production_industrial = m.addVars(industrial_users, vtype=GRB.BINARY, name="binary_production_industrial")
binary_vars_consumption_industrial = m.addVars(industrial_users, vtype=GRB.BINARY, name="binary_consumption_industrial")

binary_vars_production_commercial = m.addVars(commercial_users, vtype=GRB.BINARY, name="binary_production_commercial")
binary_vars_consumption_commercial = m.addVars(commercial_users, vtype=GRB.BINARY, name="binary_consumption_commercial")

# Auxiliary variable for absolute deviation
deviation = m.addVars(yearlist, lb=0, vtype=GRB.CONTINUOUS, name="deviation")

# Penalty coefficient for not selecting users
penalty_coefficient = 5  # Adjusted to be less aggressive

# Pre-compute production terms for each user and hour
residential_prod_terms = {
    t: {user: Residential_user_production_df.loc[t, user] for user in residential_users if user in Residential_user_production_df.columns}
    for t in yearlist
}
industrial_prod_terms = {
    t: {user: Industrial_user_production_df.loc[t, user] for user in industrial_users if user in Industrial_user_production_df.columns}
    for t in yearlist
}
commercial_prod_terms = {
    t: {user: Commercial_user_production_df.loc[t, user] for user in commercial_users if user in Commercial_user_production_df.columns}
    for t in yearlist
}

# Pre-compute consumption terms for each user and hour
residential_cons_terms = {
    t: {user: Residential_Consumption[user].loc[t] * residential_area_dict[user]
        if t in Residential_Consumption.index else 0  # Check if the index exists
        for user in residential_users}
    for t in yearlist
}

industrial_cons_terms = {
    t: {user: Industrial_Consumption[user].loc[t] * industrial_area_dict[user]
        if t in Industrial_Consumption.index else 0  # Check if the index exists
        for user in industrial_users}
    for t in yearlist
}

commercial_cons_terms = {
    t: {user: Commercial_Consumption[user].loc[t] * commercial_area_dict[user]
        if t in Commercial_Consumption.index else 0  # Check if the index exists
        for user in commercial_users}
    for t in yearlist
}

# Calculate the difference between production and consumption for each hour using precomputed terms
for t in yearlist:
    production_res = gp.quicksum(binary_vars_production_residential[user] * residential_prod_terms[t].get(user, 0) for user in residential_users)
    consumption_res = gp.quicksum(binary_vars_consumption_residential[user] * residential_cons_terms[t].get(user, 0) for user in residential_users)
    
    production_ind = gp.quicksum(binary_vars_production_industrial[user] * industrial_prod_terms[t].get(user, 0) for user in industrial_users)
    consumption_ind = gp.quicksum(binary_vars_consumption_industrial[user] * industrial_cons_terms[t].get(user, 0) for user in industrial_users)
    
    production_com = gp.quicksum(binary_vars_production_commercial[user] * commercial_prod_terms[t].get(user, 0) for user in commercial_users)
    consumption_com = gp.quicksum(binary_vars_consumption_commercial[user] * commercial_cons_terms[t].get(user, 0) for user in commercial_users)
    
    difference = production_res - consumption_res + production_ind - consumption_ind + production_com - consumption_com
    
    # Add constraints for deviation
    m.addConstr(deviation[t] >= difference, name=f"PosDeviation_{t}")
    m.addConstr(deviation[t] >= -difference, name=f"NegDeviation_{t}")

# Constraint: Total selected consumption area must equal or exceed the fixed value
m.addConstr(
    gp.quicksum(binary_vars_consumption_residential[user] * residential_area_dict[user] for user in residential_users) + 
    gp.quicksum(binary_vars_consumption_industrial[user] * industrial_area_dict[user] for user in industrial_users) +
    gp.quicksum(binary_vars_consumption_commercial[user] * commercial_area_dict[user] for user in commercial_users) >= Total_Fixed_Consumption_Area,
    name="FixedConsumptionArea"
)


# Add diversification constraint based on the total number of selected production and consumption users
diversification_coefficient = 0.5  # Set the diversification coefficient to 0.5

# New Diversification constraint: Total selected production users >= diversification_coefficient * Total selected consumption users
total_selected_production_users = (
    gp.quicksum(binary_vars_production_residential[user] for user in residential_users) +
    gp.quicksum(binary_vars_production_industrial[user] for user in industrial_users) +
    gp.quicksum(binary_vars_production_commercial[user] for user in commercial_users)
)

total_selected_consumption_users = (
    gp.quicksum(binary_vars_consumption_residential[user] for user in residential_users) +
    gp.quicksum(binary_vars_consumption_industrial[user] for user in industrial_users) +
    gp.quicksum(binary_vars_consumption_commercial[user] for user in commercial_users)
)

# Add diversification constraint
m.addConstr(total_selected_production_users >= diversification_coefficient * total_selected_consumption_users, name="DiversificationConstraint")

# Objective function: minimize the deviation and penalize unselected users
penalty = (
    penalty_coefficient * (
        gp.quicksum(1 - binary_vars_production_residential[user] for user in residential_users) +
        gp.quicksum(1 - binary_vars_consumption_residential[user] for user in residential_users) +
        gp.quicksum(1 - binary_vars_production_industrial[user] for user in industrial_users) +
        gp.quicksum(1 - binary_vars_consumption_industrial[user] for user in industrial_users) +
        gp.quicksum(1 - binary_vars_production_commercial[user] for user in commercial_users) +
        gp.quicksum(1 - binary_vars_consumption_commercial[user] for user in commercial_users)
    )
)

# Minimize deviation and add penalization
m.setObjective(gp.quicksum(deviation[t] for t in yearlist) + penalty, GRB.MINIMIZE)

# Set solver parameters to potentially reduce runtime and monitor output
m.Params.OutputFlag = 1  # Ensure detailed output
m.Params.MIPGap = 0.01  # Set gap tolerance to 1% to allow faster convergence
m.Params.TimeLimit = 10800  # Set a time limit of 1.5 hours (10800 seconds)
m.Params.Presolve = 2  # Aggressive presolve level
m.Params.Cuts = 2  # Apply aggressive cuts
m.Params.Heuristics = 0.5  # Increase the emphasis on heuristics to find feasible solutions faster

# Enable logging to file for detailed analysis later
m.Params.LogFile = os.path.join(results_dir, "optimization_log.txt")

# Optimize the model
m.optimize()

# Check for solution status
if m.Status == GRB.OPTIMAL:
    print("Optimal solution found!")
elif m.Status == GRB.INFEASIBLE:
    print("Model is infeasible.")
    m.computeIIS()  # Compute Irreducible Inconsistent Subsystem (IIS) to understand infeasibility
    m.write(os.path.join(results_dir, "model.ilp"))  # Save IIS to a file
elif m.Status == GRB.TIME_LIMIT:
    print("Optimization stopped due to time limit.")
else:
    print(f"Optimization ended with status {m.Status}.")

# Display the results
for v in m.getVars():
    print(f'{v.varName}: {v.x}')

print(f'Objective: {m.objVal}')

# Extract results for each user type
RP_values = {user: binary_vars_production_residential[user].X for user in residential_users}
CP_values = {user: binary_vars_consumption_residential[user].X for user in residential_users}
IP_values = {user: binary_vars_production_industrial[user].X for user in industrial_users}
CIP_values = {user: binary_vars_consumption_industrial[user].X for user in industrial_users}
CP_values_com = {user: binary_vars_production_commercial[user].X for user in commercial_users}
CIP_values_com = {user: binary_vars_consumption_commercial[user].X for user in commercial_users}

# Create new DataFrames with the chosen binary variables
DF_m2_Residential['Chosen_Production'] = [RP_values[user] for user in DF_m2_Residential['full_id']]
DF_m2_Residential['Chosen_Consumption'] = [CP_values[user] for user in DF_m2_Residential['full_id']]
DF_m2_Industrial['Chosen_Production'] = [IP_values[user] for user in DF_m2_Industrial['full_id']]
DF_m2_Industrial['Chosen_Consumption'] = [CIP_values[user] for user in DF_m2_Industrial['full_id']]
DF_m2_Commercial['Chosen_Production'] = [CP_values_com[user] for user in DF_m2_Commercial['full_id']]
DF_m2_Commercial['Chosen_Consumption'] = [CIP_values_com[user] for user in DF_m2_Commercial['full_id']]

# Save the results to CSV files
DF_m2_Residential.to_csv(os.path.join(results_dir, 'New_residential_DF.csv'))
DF_m2_Industrial.to_csv(os.path.join(results_dir, 'New_Industrial_DF.csv'))
DF_m2_Commercial.to_csv(os.path.join(results_dir, 'New_commercial_DF.csv'))

# Calculate the number of chosen users versus the total number of users for each category
num_residential_chosen_production = len(DF_m2_Residential[DF_m2_Residential['Chosen_Production'] == 1])
num_residential_total = len(DF_m2_Residential)
num_residential_chosen_consumption = len(DF_m2_Residential[DF_m2_Residential['Chosen_Consumption'] == 1])

num_industrial_chosen_production = len(DF_m2_Industrial[DF_m2_Industrial['Chosen_Production'] == 1])
num_industrial_total = len(DF_m2_Industrial)
num_industrial_chosen_consumption = len(DF_m2_Industrial[DF_m2_Industrial['Chosen_Consumption'] == 1])

num_commercial_chosen_production = len(DF_m2_Commercial[DF_m2_Commercial['Chosen_Production'] == 1])
num_commercial_total = len(DF_m2_Commercial)
num_commercial_chosen_consumption = len(DF_m2_Commercial[DF_m2_Commercial['Chosen_Consumption'] == 1])

# Print the amounts
print(f'Residential Production: Chosen = {num_residential_chosen_production}, Total = {num_residential_total}')
print(f'Residential Consumption: Chosen = {num_residential_chosen_consumption}, Total = {num_residential_total}')
print(f'Industrial Production: Chosen = {num_industrial_chosen_production}, Total = {num_industrial_total}')
print(f'Industrial Consumption: Chosen = {num_industrial_chosen_consumption}, Total = {num_industrial_total}')
print(f'Commercial Production: Chosen = {num_commercial_chosen_production}, Total = {num_commercial_total}')
print(f'Commercial Consumption: Chosen = {num_commercial_chosen_consumption}, Total = {num_commercial_total}')

# Summary of Results
print("\nSummary of Optimization Results:")
print(f"Total Residential Production Users Chosen: {num_residential_chosen_production} out of {num_residential_total} ({(num_residential_chosen_production / num_residential_total) * 100:.2f}%)")
print(f"Total Residential Consumption Users Chosen: {num_residential_chosen_consumption} out of {num_residential_total} ({(num_residential_chosen_consumption / num_residential_total) * 100:.2f}%)")

print(f"Total Industrial Production Users Chosen: {num_industrial_chosen_production} out of {num_industrial_total} ({(num_industrial_chosen_production / num_industrial_total) * 100:.2f}%)")
print(f"Total Industrial Consumption Users Chosen: {num_industrial_chosen_consumption} out of {num_industrial_total} ({(num_industrial_chosen_consumption / num_industrial_total) * 100:.2f}%)")

print(f"Total Commercial Production Users Chosen: {num_commercial_chosen_production} out of {num_commercial_total} ({(num_commercial_chosen_production / num_commercial_total) * 100:.2f}%)")
print(f"Total Commercial Consumption Users Chosen: {num_commercial_chosen_consumption} out of {num_commercial_total} ({(num_commercial_chosen_consumption / num_commercial_total) * 100:.2f}%)")


## No penalty in the code

In [ ]:
import pandas as pd
import os
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from datetime import datetime

# Paths to directories
Data_path_GDrive = "G:/Il mio Drive/Thesis_Large_Files/Working Table"
Data_path_Production = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Production"
Data_path_Consumption = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption"
Data_path_Users = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Users"
results_dir = 'C:/Users/Nik/Documents/GitHub/Thesis/Results/Results_Binary/Final Results'
results_dir = os.path.join(results_dir, datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + 'BinaryProductionandConsumption')
os.makedirs(results_dir, exist_ok=True)

# Load Working Tables for every optimization try
WorkingTables_Path = 'C:/Users/Nik/Documents/GitHub/Thesis/CSV/Working Tables'
DF_m2_Residential = pd.read_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\ResidentialConsumption-4000\Modified_DF_m2_Residential.csv', sep=",", on_bad_lines='skip', header=0)
DF_m2_Industrial = pd.read_csv(os.path.join(WorkingTables_Path, 'DF_m2_Industrial.csv'), sep=",", on_bad_lines='skip', header=0)
DF_m2_Commercial = pd.read_csv(os.path.join(WorkingTables_Path, 'DF_m2_Commercial.csv'), sep=",", on_bad_lines='skip', header=0)

Residential_user_production_df = pd.read_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\ResidentialConsumption-4000\Modified_Residential_Production.csv', sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])
Industrial_user_production_df = pd.read_csv(os.path.join(WorkingTables_Path, 'Industrial_user_production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])
Commercial_user_production_df = pd.read_csv(os.path.join(WorkingTables_Path, 'Commercial_user_production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])

# Load New Consumption Curves
Residential_Consumption = pd.read_csv(r'C:\Users\Nik\Desktop\Backup thesis\Consumption\ResidentialConsumption-4000\Modified_Residential_Consumption.csv', index_col=0, parse_dates=True)
Industrial_Consumption = pd.read_csv('C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption/Industrial_Consumption.csv', index_col=0, parse_dates=True)
Commercial_Consumption = pd.read_csv('C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption/Commercial_Consumption.csv', index_col=0, parse_dates=True)

DF_Production = pd.read_csv(os.path.join(Data_path_Production, '2019_Production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])
print("ok")
# Ensure indices are in datetime format for alignment
Residential_Consumption.index = pd.to_datetime(Residential_Consumption.index)
Industrial_Consumption.index = pd.to_datetime(Industrial_Consumption.index)
Commercial_Consumption.index = pd.to_datetime(Commercial_Consumption.index)
DF_Production.index = pd.to_datetime(DF_Production.index)

# Use 'full_id' as the primary identifier
residential_users = DF_m2_Residential['full_id'].astype(str).tolist()
industrial_users = DF_m2_Industrial['full_id'].astype(str).tolist()
commercial_users = DF_m2_Commercial['full_id'].astype(str).tolist()

# Precompute area mappings to avoid repeated DataFrame lookups
residential_area_dict = DF_m2_Residential.set_index('full_id')['Area'].to_dict()
industrial_area_dict = DF_m2_Industrial.set_index('full_id')['Area'].to_dict()
commercial_area_dict = DF_m2_Commercial.set_index('full_id')['Area'].to_dict()

# Define the yearlist based on the number of hours
yearlist = pd.to_datetime(DF_Production.index)

# Dynamic consumption area constraints (e.g., set as 60% of the total available area)
Fixed_Res_Consumption_Area = 0.6 * DF_m2_Residential['Area'].sum()
Fixed_Ind_Consumption_Area = 0.6 * DF_m2_Industrial['Area'].sum()
Fixed_Com_Consumption_Area = 0.6 * DF_m2_Commercial['Area'].sum()
Total_Fixed_Consumption_Area = Fixed_Res_Consumption_Area + Fixed_Ind_Consumption_Area + Fixed_Com_Consumption_Area

# Create a new Gurobi model
m = gp.Model("Optimization_Model_ConsumptionProduction")

# Production and Consumption Binary Variables [0;1]
binary_vars_production_residential = m.addVars(residential_users, vtype=GRB.BINARY, name="binary_production_residential")
binary_vars_consumption_residential = m.addVars(residential_users, vtype=GRB.BINARY, name="binary_consumption_residential")

binary_vars_production_industrial = m.addVars(industrial_users, vtype=GRB.BINARY, name="binary_production_industrial")
binary_vars_consumption_industrial = m.addVars(industrial_users, vtype=GRB.BINARY, name="binary_consumption_industrial")

binary_vars_production_commercial = m.addVars(commercial_users, vtype=GRB.BINARY, name="binary_production_commercial")
binary_vars_consumption_commercial = m.addVars(commercial_users, vtype=GRB.BINARY, name="binary_consumption_commercial")

# Auxiliary variable for absolute deviation
deviation = m.addVars(yearlist, lb=0, vtype=GRB.CONTINUOUS, name="deviation")

# Penalty coefficient for not selecting users
penalty_coefficient = 0 

# Pre-compute production terms for each user and hour
residential_prod_terms = {
    t: {user: Residential_user_production_df.loc[t, user] for user in residential_users if user in Residential_user_production_df.columns}
    for t in yearlist
}
industrial_prod_terms = {
    t: {user: Industrial_user_production_df.loc[t, user] for user in industrial_users if user in Industrial_user_production_df.columns}
    for t in yearlist
}
commercial_prod_terms = {
    t: {user: Commercial_user_production_df.loc[t, user] for user in commercial_users if user in Commercial_user_production_df.columns}
    for t in yearlist
}

# Pre-compute consumption terms for each user and hour
residential_cons_terms = {
    t: {user: Residential_Consumption[user].loc[t] * residential_area_dict[user]
        if t in Residential_Consumption.index else 0
        for user in residential_users}
    for t in yearlist
}

industrial_cons_terms = {
    t: {user: Industrial_Consumption[user].loc[t] * industrial_area_dict[user]
        if t in Industrial_Consumption.index else 0
        for user in industrial_users}
    for t in yearlist
}

commercial_cons_terms = {
    t: {user: Commercial_Consumption[user].loc[t] * commercial_area_dict[user]
        if t in Commercial_Consumption.index else 0
        for user in commercial_users}
    for t in yearlist
}

# Calculate the difference between production and consumption for each hour using precomputed terms
for t in yearlist:
    production_res = gp.quicksum(binary_vars_production_residential[user] * residential_prod_terms[t].get(user, 0) for user in residential_users)
    consumption_res = gp.quicksum(binary_vars_consumption_residential[user] * residential_cons_terms[t].get(user, 0) for user in residential_users)
    
    production_ind = gp.quicksum(binary_vars_production_industrial[user] * industrial_prod_terms[t].get(user, 0) for user in industrial_users)
    consumption_ind = gp.quicksum(binary_vars_consumption_industrial[user] * industrial_cons_terms[t].get(user, 0) for user in industrial_users)
    
    production_com = gp.quicksum(binary_vars_production_commercial[user] * commercial_prod_terms[t].get(user, 0) for user in commercial_users)
    consumption_com = gp.quicksum(binary_vars_consumption_commercial[user] * commercial_cons_terms[t].get(user, 0) for user in commercial_users)
    
    difference = production_res - consumption_res + production_ind - consumption_ind + production_com - consumption_com
    
    # Add constraints for deviation
    m.addConstr(deviation[t] >= difference, name=f"PosDeviation_{t}")
    m.addConstr(deviation[t] >= -difference, name=f"NegDeviation_{t}")

# Constraint: Total selected consumption area must equal or exceed the fixed value
m.addConstr(
    gp.quicksum(binary_vars_consumption_residential[user] * residential_area_dict[user] for user in residential_users) + 
    gp.quicksum(binary_vars_consumption_industrial[user] * industrial_area_dict[user] for user in industrial_users) +
    gp.quicksum(binary_vars_consumption_commercial[user] * commercial_area_dict[user] for user in commercial_users) >= Total_Fixed_Consumption_Area,
    name="FixedConsumptionArea"
)

# Add diversification constraints
diversification_factor = 0.5  

# Residential diversification constraint
m.addConstr(
    gp.quicksum(binary_vars_production_residential[user] for user in residential_users) >= diversification_factor * gp.quicksum(binary_vars_consumption_residential[user] for user in residential_users),
    name="ResDiversificationProductionConsumption"
)
m.addConstr(
    gp.quicksum(binary_vars_consumption_residential[user] for user in residential_users) >= diversification_factor * gp.quicksum(binary_vars_production_residential[user] for user in residential_users),
    name="ResDiversificationConsumptionProduction"
)

# Industrial diversification constraint
m.addConstr(
    gp.quicksum(binary_vars_production_industrial[user] for user in industrial_users) >= diversification_factor * gp.quicksum(binary_vars_consumption_industrial[user] for user in industrial_users),
    name="IndDiversificationProductionConsumption"
)
m.addConstr(
    gp.quicksum(binary_vars_consumption_industrial[user] for user in industrial_users) >= diversification_factor * gp.quicksum(binary_vars_production_industrial[user] for user in industrial_users),
    name="IndDiversificationConsumptionProduction"
)

# Commercial diversification constraint
m.addConstr(
    gp.quicksum(binary_vars_production_commercial[user] for user in commercial_users) >= diversification_factor * gp.quicksum(binary_vars_consumption_commercial[user] for user in commercial_users),
    name="ComDiversificationProductionConsumption"
)
m.addConstr(
    gp.quicksum(binary_vars_consumption_commercial[user] for user in commercial_users) >= diversification_factor * gp.quicksum(binary_vars_production_commercial[user] for user in commercial_users),
    name="ComDiversificationConsumptionProduction"
)

# Objective function: minimize the deviation and penalize unselected users
penalty = (
    penalty_coefficient * (
        gp.quicksum(1 - binary_vars_production_residential[user] for user in residential_users) +
        gp.quicksum(1 - binary_vars_consumption_residential[user] for user in residential_users) +
        gp.quicksum(1 - binary_vars_production_industrial[user] for user in industrial_users) +
        gp.quicksum(1 - binary_vars_consumption_industrial[user] for user in industrial_users) +
        gp.quicksum(1 - binary_vars_production_commercial[user] for user in commercial_users) +
        gp.quicksum(1 - binary_vars_consumption_commercial[user] for user in commercial_users)
    )
)

# Minimize deviation and add penalization
m.setObjective(gp.quicksum(deviation[t] for t in yearlist) + penalty, GRB.MINIMIZE)

# Set solver parameters to potentially reduce runtime and monitor output
m.Params.OutputFlag = 1  # Ensure detailed output
m.Params.MIPGap = 0.01  # Set gap tolerance to 1% to allow faster convergence
m.Params.TimeLimit = 10800  # Set a time limit of 3 hours (10800 seconds)
m.Params.Presolve = 2  # Aggressive presolve level
m.Params.Cuts = 2  # Apply aggressive cuts
m.Params.Heuristics = 0.5  # Increase the emphasis on heuristics to find feasible solutions faster

# Enable logging to file for detailed analysis later
m.Params.LogFile = os.path.join(results_dir, "optimization_log.txt")

# Optimize the model
m.optimize()

# Check for solution status
if m.Status == GRB.OPTIMAL:
    print("Optimal solution found!")
elif m.Status == GRB.INFEASIBLE:
    print("Model is infeasible.")
    m.computeIIS()  # Compute Irreducible Inconsistent Subsystem (IIS) to understand infeasibility
    m.write(os.path.join(results_dir, "model.ilp"))  # Save IIS to a file
elif m.Status == GRB.TIME_LIMIT:
    print("Optimization stopped due to time limit.")
else:
    print(f"Optimization ended with status {m.Status}.")

# Display the results
for v in m.getVars():
    print(f'{v.varName}: {v.x}')

print(f'Objective: {m.objVal}')

# Extract results for each user type and create DataFrames for chosen users
DF_m2_Residential['Chosen_Production'] = [binary_vars_production_residential[user].X for user in DF_m2_Residential['full_id']]
DF_m2_Residential['Chosen_Consumption'] = [binary_vars_consumption_residential[user].X for user in DF_m2_Residential['full_id']]
DF_m2_Industrial['Chosen_Production'] = [binary_vars_production_industrial[user].X for user in DF_m2_Industrial['full_id']]
DF_m2_Industrial['Chosen_Consumption'] = [binary_vars_consumption_industrial[user].X for user in DF_m2_Industrial['full_id']]
DF_m2_Commercial['Chosen_Production'] = [binary_vars_production_commercial[user].X for user in DF_m2_Commercial['full_id']]
DF_m2_Commercial['Chosen_Consumption'] = [binary_vars_consumption_commercial[user].X for user in DF_m2_Commercial['full_id']]

# Save the results to CSV files
DF_m2_Residential.to_csv(os.path.join(results_dir, 'New_residential_DF.csv'))
DF_m2_Industrial.to_csv(os.path.join(results_dir, 'New_Industrial_DF.csv'))
DF_m2_Commercial.to_csv(os.path.join(results_dir, 'New_commercial_DF.csv'))

# Calculate the number of chosen users versus the total number of users for each category
num_residential_chosen_production = len(DF_m2_Residential[DF_m2_Residential['Chosen_Production'] == 1])
num_residential_total = len(DF_m2_Residential)
num_residential_chosen_consumption = len(DF_m2_Residential[DF_m2_Residential['Chosen_Consumption'] == 1])

num_industrial_chosen_production = len(DF_m2_Industrial[DF_m2_Industrial['Chosen_Production'] == 1])
num_industrial_total = len(DF_m2_Industrial)
num_industrial_chosen_consumption = len(DF_m2_Industrial[DF_m2_Industrial['Chosen_Consumption'] == 1])

num_commercial_chosen_production = len(DF_m2_Commercial[DF_m2_Commercial['Chosen_Production'] == 1])
num_commercial_total = len(DF_m2_Commercial)
num_commercial_chosen_consumption = len(DF_m2_Commercial[DF_m2_Commercial['Chosen_Consumption'] == 1])

# Print the amounts
print(f'Residential Production: Chosen = {num_residential_chosen_production}, Total = {num_residential_total}')
print(f'Residential Consumption: Chosen = {num_residential_chosen_consumption}, Total = {num_residential_total}')
print(f'Industrial Production: Chosen = {num_industrial_chosen_production}, Total = {num_industrial_total}')
print(f'Industrial Consumption: Chosen = {num_industrial_chosen_consumption}, Total = {num_industrial_total}')
print(f'Commercial Production: Chosen = {num_commercial_chosen_production}, Total = {num_commercial_total}')
print(f'Commercial Consumption: Chosen = {num_commercial_chosen_consumption}, Total = {num_commercial_total}')

# Summary of Results
print("\nSummary of Optimization Results:")
print(f"Total Residential Production Users Chosen: {num_residential_chosen_production} out of {num_residential_total} ({(num_residential_chosen_production / num_residential_total) * 100:.2f}%)")
print(f"Total Residential Consumption Users Chosen: {num_residential_chosen_consumption} out of {num_residential_total} ({(num_residential_chosen_consumption / num_residential_total) * 100:.2f}%)")

print(f"Total Industrial Production Users Chosen: {num_industrial_chosen_production} out of {num_industrial_total} ({(num_industrial_chosen_production / num_industrial_total) * 100:.2f}%)")
print(f"Total Industrial Consumption Users Chosen: {num_industrial_chosen_consumption} out of {num_industrial_total} ({(num_industrial_chosen_consumption / num_industrial_total) * 100:.2f}%)")

print(f"Total Commercial Production Users Chosen: {num_commercial_chosen_production} out of {num_commercial_total} ({(num_commercial_chosen_production / num_commercial_total) * 100:.2f}%)")
print(f"Total Commercial Consumption Users Chosen: {num_commercial_chosen_consumption} out of {num_commercial_total} ({(num_commercial_chosen_consumption / num_commercial_total) * 100:.2f}%)")

# Check if production and consumption constraints are met
residential_prod_area = DF_m2_Residential[DF_m2_Residential['Chosen_Production'] == 1]['Area'].sum()
residential_cons_area = DF_m2_Residential[DF_m2_Residential['Chosen_Consumption'] == 1]['Area'].sum()

industrial_prod_area = DF_m2_Industrial[DF_m2_Industrial['Chosen_Production'] == 1]['Area'].sum()
industrial_cons_area = DF_m2_Industrial[DF_m2_Industrial['Chosen_Consumption'] == 1]['Area'].sum()

commercial_prod_area = DF_m2_Commercial[DF_m2_Commercial['Chosen_Production'] == 1]['Area'].sum()
commercial_cons_area = DF_m2_Commercial[DF_m2_Commercial['Chosen_Consumption'] == 1]['Area'].sum()

print(f"\nResidential Production Area: {residential_prod_area:.2f} | Consumption Area: {residential_cons_area:.2f} (Required >= {Fixed_Res_Consumption_Area:.2f})")
print(f"Industrial Production Area: {industrial_prod_area:.2f} | Consumption Area: {industrial_cons_area:.2f} (Required >= {Fixed_Ind_Consumption_Area:.2f})")
print(f"Commercial Production Area: {commercial_prod_area:.2f} | Consumption Area: {commercial_cons_area:.2f} (Required >= {Fixed_Com_Consumption_Area:.2f})")

# Check if diversification constraints are met
print("\nDiversification Checks:")
residential_prod_count = len(DF_m2_Residential[DF_m2_Residential['Chosen_Production'] == 1])
residential_cons_count = len(DF_m2_Residential[DF_m2_Residential['Chosen_Consumption'] == 1])

industrial_prod_count = len(DF_m2_Industrial[DF_m2_Industrial['Chosen_Production'] == 1])
industrial_cons_count = len(DF_m2_Industrial[DF_m2_Industrial['Chosen_Consumption'] == 1])

commercial_prod_count = len(DF_m2_Commercial[DF_m2_Commercial['Chosen_Production'] == 1])
commercial_cons_count = len(DF_m2_Commercial[DF_m2_Commercial['Chosen_Consumption'] == 1])

print(f"Residential Production Users Chosen: {residential_prod_count} | Consumption Users Chosen: {residential_cons_count}")
print(f"Industrial Production Users Chosen: {industrial_prod_count} | Consumption Users Chosen: {industrial_cons_count}")
print(f"Commercial Production Users Chosen: {commercial_prod_count} | Consumption Users Chosen: {commercial_cons_count}")

# Ensure the diversification factor is respected
if residential_prod_count >= diversification_factor * residential_cons_count and residential_cons_count >= diversification_factor * residential_prod_count:
    print("Residential diversification constraint is satisfied.")
else:
    print("Residential diversification constraint is NOT satisfied.")

if industrial_prod_count >= diversification_factor * industrial_cons_count and industrial_cons_count >= diversification_factor * industrial_prod_count:
    print("Industrial diversification constraint is satisfied.")
else:
    print("Industrial diversification constraint is NOT satisfied.")

if commercial_prod_count >= diversification_factor * commercial_cons_count and commercial_cons_count >= diversification_factor * commercial_prod_count:
    print("Commercial diversification constraint is satisfied.")
else:
    print("Commercial diversification constraint is NOT satisfied.")

In [5]:
# Calculate the number of chosen users versus the total number of users for each category
num_residential_chosen_production = len(DF_m2_Residential[DF_m2_Residential['Chosen_Production'] == 1])
num_residential_total = len(DF_m2_Residential)
num_residential_chosen_consumption = len(DF_m2_Residential[DF_m2_Residential['Chosen_Consumption'] == 1])

num_industrial_chosen_production = len(DF_m2_Industrial[DF_m2_Industrial['Chosen_Production'] == 1])
num_industrial_total = len(DF_m2_Industrial)
num_industrial_chosen_consumption = len(DF_m2_Industrial[DF_m2_Industrial['Chosen_Consumption'] == 1])

num_commercial_chosen_production = len(DF_m2_Commercial[DF_m2_Commercial['Chosen_Production'] == 1])
num_commercial_total = len(DF_m2_Commercial)
num_commercial_chosen_consumption = len(DF_m2_Commercial[DF_m2_Commercial['Chosen_Consumption'] == 1])

# Print the amounts
print(f'Residential Production: Chosen = {num_residential_chosen_production}, Total = {num_residential_total}')
print(f'Residential Consumption: Chosen = {num_residential_chosen_consumption}, Total = {num_residential_total}')
print(f'Industrial Production: Chosen = {num_industrial_chosen_production}, Total = {num_industrial_total}')
print(f'Industrial Consumption: Chosen = {num_industrial_chosen_consumption}, Total = {num_industrial_total}')
print(f'Commercial Production: Chosen = {num_commercial_chosen_production}, Total = {num_commercial_total}')
print(f'Commercial Consumption: Chosen = {num_commercial_chosen_consumption}, Total = {num_commercial_total}')

# Summary of Results
print("\nSummary of Optimization Results:")
print(f"Total Residential Production Users Chosen: {num_residential_chosen_production} out of {num_residential_total} ({(num_residential_chosen_production / num_residential_total) * 100:.2f}%)")
print(f"Total Residential Consumption Users Chosen: {num_residential_chosen_consumption} out of {num_residential_total} ({(num_residential_chosen_consumption / num_residential_total) * 100:.2f}%)")

print(f"Total Industrial Production Users Chosen: {num_industrial_chosen_production} out of {num_industrial_total} ({(num_industrial_chosen_production / num_industrial_total) * 100:.2f}%)")
print(f"Total Industrial Consumption Users Chosen: {num_industrial_chosen_consumption} out of {num_industrial_total} ({(num_industrial_chosen_consumption / num_industrial_total) * 100:.2f}%)")

print(f"Total Commercial Production Users Chosen: {num_commercial_chosen_production} out of {num_commercial_total} ({(num_commercial_chosen_production / num_commercial_total) * 100:.2f}%)")
print(f"Total Commercial Consumption Users Chosen: {num_commercial_chosen_consumption} out of {num_commercial_total} ({(num_commercial_chosen_consumption / num_commercial_total) * 100:.2f}%)")

# Check if production and consumption constraints are met
residential_prod_area = DF_m2_Residential[DF_m2_Residential['Chosen_Production'] == 1]['Area'].sum()
residential_cons_area = DF_m2_Residential[DF_m2_Residential['Chosen_Consumption'] == 1]['Area'].sum()

industrial_prod_area = DF_m2_Industrial[DF_m2_Industrial['Chosen_Production'] == 1]['Area'].sum()
industrial_cons_area = DF_m2_Industrial[DF_m2_Industrial['Chosen_Consumption'] == 1]['Area'].sum()

commercial_prod_area = DF_m2_Commercial[DF_m2_Commercial['Chosen_Production'] == 1]['Area'].sum()
commercial_cons_area = DF_m2_Commercial[DF_m2_Commercial['Chosen_Consumption'] == 1]['Area'].sum()

print(f"\nResidential Production Area: {residential_prod_area:.2f} | Consumption Area: {residential_cons_area:.2f} (Required >= {Fixed_Res_Consumption_Area:.2f})")
print(f"Industrial Production Area: {industrial_prod_area:.2f} | Consumption Area: {industrial_cons_area:.2f} (Required >= {Fixed_Ind_Consumption_Area:.2f})")
print(f"Commercial Production Area: {commercial_prod_area:.2f} | Consumption Area: {commercial_cons_area:.2f} (Required >= {Fixed_Com_Consumption_Area:.2f})")

# Check if diversification constraints are met
print("\nDiversification Checks:")
residential_prod_count = len(DF_m2_Residential[DF_m2_Residential['Chosen_Production'] == 1])
residential_cons_count = len(DF_m2_Residential[DF_m2_Residential['Chosen_Consumption'] == 1])

industrial_prod_count = len(DF_m2_Industrial[DF_m2_Industrial['Chosen_Production'] == 1])
industrial_cons_count = len(DF_m2_Industrial[DF_m2_Industrial['Chosen_Consumption'] == 1])

commercial_prod_count = len(DF_m2_Commercial[DF_m2_Commercial['Chosen_Production'] == 1])
commercial_cons_count = len(DF_m2_Commercial[DF_m2_Commercial['Chosen_Consumption'] == 1])

print(f"Residential Production Users Chosen: {residential_prod_count} | Consumption Users Chosen: {residential_cons_count}")
print(f"Industrial Production Users Chosen: {industrial_prod_count} | Consumption Users Chosen: {industrial_cons_count}")
print(f"Commercial Production Users Chosen: {commercial_prod_count} | Consumption Users Chosen: {commercial_cons_count}")
# Ensure the diversification factor is respected
if residential_prod_count >= diversification_factor * residential_cons_count and residential_cons_count >= diversification_factor * residential_prod_count:
    print("Residential diversification constraint is satisfied.")
else:
    print("Residential diversification constraint is NOT satisfied.")

if industrial_prod_count >= diversification_factor * industrial_cons_count and industrial_cons_count >= diversification_factor * industrial_prod_count:
    print("Industrial diversification constraint is satisfied.")
else:
    print("Industrial diversification constraint is NOT satisfied.")

if commercial_prod_count >= diversification_factor * commercial_cons_count and commercial_cons_count >= diversification_factor * commercial_prod_count:
    print("Commercial diversification constraint is satisfied.")
else:
    print("Commercial diversification constraint is NOT satisfied.")

Residential Production: Chosen = 998, Total = 3837
Residential Consumption: Chosen = 2117, Total = 3837
Industrial Production: Chosen = 18, Total = 60
Industrial Consumption: Chosen = 41, Total = 60
Commercial Production: Chosen = 75, Total = 265
Commercial Consumption: Chosen = 15, Total = 265

Summary of Optimization Results:
Total Residential Production Users Chosen: 998 out of 3837 (26.01%)
Total Residential Consumption Users Chosen: 2117 out of 3837 (55.17%)
Total Industrial Production Users Chosen: 18 out of 60 (30.00%)
Total Industrial Consumption Users Chosen: 41 out of 60 (68.33%)
Total Commercial Production Users Chosen: 75 out of 265 (28.30%)
Total Commercial Consumption Users Chosen: 15 out of 265 (5.66%)

Residential Production Area: 22226.46 | Consumption Area: 70157.12 (Required >= 52215.57)
Industrial Production Area: 1717.06 | Consumption Area: 6259.11 (Required >= 4210.23)
Commercial Production Area: 14118.79 | Consumption Area: 11368.06 (Required >= 31358.40)

Divers

NameError: name 'diversification_factor' is not defined

In [4]:
import matplotlib.pyplot as plt

# Data for plotting
categories = ['Residential', 'Industrial', 'Commercial']
chosen_production = [num_residential_chosen_production, num_industrial_chosen_production, num_commercial_chosen_production]
total_users = [num_residential_total, num_industrial_total, num_commercial_total]
chosen_consumption = [num_residential_chosen_consumption, num_industrial_chosen_consumption, num_commercial_chosen_consumption]

# Plotting Chosen vs Total Users for Production
plt.figure(figsize=(10, 6))
bar_width = 0.35
x = np.arange(len(categories))

plt.bar(x, total_users, width=bar_width, label='Total Users', alpha=0.7)
plt.bar(x + bar_width, chosen_production, width=bar_width, label='Chosen for Production', alpha=0.7)

plt.xlabel('Category')
plt.ylabel('Number of Users')
plt.title('Chosen vs Total Users for Production')
plt.xticks(x + bar_width / 2, categories)
plt.legend()
plt.show()

# Plotting Chosen vs Total Users for Consumption
plt.figure(figsize=(10, 6))

plt.bar(x, total_users, width=bar_width, label='Total Users', alpha=0.7)
plt.bar(x + bar_width, chosen_consumption, width=bar_width, label='Chosen for Consumption', alpha=0.7)

plt.xlabel('Category')
plt.ylabel('Number of Users')
plt.title('Chosen vs Total Users for Consumption')
plt.xticks(x + bar_width / 2, categories)
plt.legend()
plt.show()

NameError: name 'num_residential_chosen_production' is not defined

In [ ]:
import matplotlib.pyplot as plt
# Use data from chosen users for production and consumption areas
residential_production_areas = chosen_residential_production['Area'].values
industrial_production_areas = chosen_industrial_production['Area'].values
commercial_production_areas = chosen_commercial_production['Area'].values

residential_consumption_areas = chosen_residential_consumption['Area'].values
industrial_consumption_areas = chosen_industrial_consumption['Area'].values
commercial_consumption_areas = chosen_commercial_consumption['Area'].values

plt.figure(figsize=(18, 10))

# Residential Production
plt.subplot(2, 3, 1)
plt.hist(residential_production_areas, bins=50, alpha=0.6, color='blue')
plt.xlabel('Area', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Residential Production Areas', fontsize=14)
plt.xlim(0, max(residential_production_areas))  # Set x-axis limit to max value
plt.yscale('log')  # Use logarithmic scale for y-axis
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Residential Consumption
plt.subplot(2, 3, 4)
plt.hist(residential_consumption_areas, bins=50, alpha=0.6, color='blue')
plt.xlabel('Area', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Residential Consumption Areas', fontsize=14)
plt.xlim(0, max(residential_consumption_areas))  # Set x-axis limit to max value
plt.yscale('log')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Industrial Production
plt.subplot(2, 3, 2)
plt.hist(industrial_production_areas, bins=50, alpha=0.6, color='green')
plt.xlabel('Area', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Industrial Production Areas', fontsize=14)
plt.xlim(0, max(industrial_production_areas))  # Set x-axis limit to max value
plt.yscale('log')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Industrial Consumption
plt.subplot(2, 3, 5)
plt.hist(industrial_consumption_areas, bins=50, alpha=0.6, color='green')
plt.xlabel('Area', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Industrial Consumption Areas', fontsize=14)
plt.xlim(0, max(industrial_consumption_areas))  # Set x-axis limit to max value
plt.yscale('log')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Commercial Production
plt.subplot(2, 3, 3)
plt.hist(commercial_production_areas, bins=50, alpha=0.6, color='orange')
plt.xlabel('Area', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Commercial Production Areas', fontsize=14)
plt.xlim(0, max(commercial_production_areas))  # Set x-axis limit to max value
plt.yscale('log')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Commercial Consumption
plt.subplot(2, 3, 6)
plt.hist(commercial_consumption_areas, bins=50, alpha=0.6, color='orange')
plt.xlabel('Area', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Commercial Consumption Areas', fontsize=14)
plt.xlim(0, max(commercial_consumption_areas))  # Set x-axis limit to max value
plt.yscale('log')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

plt.tight_layout()
plt.show()


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Ensure all production and consumption data indices are in datetime format
Residential_user_production_df.index = pd.to_datetime(Residential_user_production_df.index)
Industrial_user_production_df.index = pd.to_datetime(Industrial_user_production_df.index)
Commercial_user_production_df.index = pd.to_datetime(Commercial_user_production_df.index)

Residential_Consumption.index = pd.to_datetime(Residential_Consumption.index)
Industrial_Consumption.index = pd.to_datetime(Industrial_Consumption.index)
Commercial_Consumption.index = pd.to_datetime(Commercial_Consumption.index)

# Set 'full_id' as the index for the DF_m2_* DataFrames if it is not already the index
if 'full_id' in DF_m2_Residential.columns:
    DF_m2_Residential.set_index('full_id', inplace=True)

if 'full_id' in DF_m2_Industrial.columns:
    DF_m2_Industrial.set_index('full_id', inplace=True)

if 'full_id' in DF_m2_Commercial.columns:
    DF_m2_Commercial.set_index('full_id', inplace=True)


# Strip any extra spaces in 'full_id' and production DataFrame column names
DF_m2_Residential.index = DF_m2_Residential.index.astype(str).str.strip()
DF_m2_Industrial.index = DF_m2_Industrial.index.astype(str).str.strip()
DF_m2_Commercial.index = DF_m2_Commercial.index.astype(str).str.strip()

Residential_user_production_df.columns = Residential_user_production_df.columns.astype(str).str.strip()
Industrial_user_production_df.columns = Industrial_user_production_df.columns.astype(str).str.strip()
Commercial_user_production_df.columns = Commercial_user_production_df.columns.astype(str).str.strip()

# Initialize directories for saving plots
plot_dir = os.path.join(results_dir, 'Plot')
weekly_plot_dir = os.path.join(plot_dir, 'Weekly_Plot')
os.makedirs(weekly_plot_dir, exist_ok=True)

# Initialize a dictionary to store the results
weekly_results = {
    'Month': [],
    'F1_Percentage': [],
    'F2_Percentage': [],
    'F3_Percentage': [],
    '10am_to_4pm_Percentage': []
}

# Print a sample of identifiers for debugging
print(f"Sample Residential full_id in DF_m2_Residential: {DF_m2_Residential.index[:5].tolist()}")
print(f"Sample columns in Residential_user_production_df: {Residential_user_production_df.columns[:5].tolist()}")

# Function to filter users that are present in the DataFrame columns for either production or consumption
def filter_valid_users(df, data_df, chosen_column):
    # Filter users based on their presence in the DataFrame and if they are chosen (1 in the chosen_column)
    valid_users = [user for user in df.index if user in data_df.columns and df.loc[user, chosen_column] == 1]
    print(f"Valid users for {chosen_column}: {valid_users[:5]}...")  # Debug: print first 5 valid users
    return valid_users

# Loop through each month to calculate percentages for each period and plot the first week of each month
for month in range(1, 13):
    start_date = f'2019-{month:02d}-01'
    end_date = f'2019-{month:02d}-07'

    # Filter data for the first week of the month
    week_data_index = pd.date_range(start=start_date, end=end_date, freq='H')

    # Filter valid production users based on their presence in production data
    valid_residential_production_users = filter_valid_users(DF_m2_Residential, Residential_user_production_df, 'Chosen_Production')
    valid_industrial_production_users = filter_valid_users(DF_m2_Industrial, Industrial_user_production_df, 'Chosen_Production')
    valid_commercial_production_users = filter_valid_users(DF_m2_Commercial, Commercial_user_production_df, 'Chosen_Production')

    # Filter valid consumption users based on their presence in consumption data
    valid_residential_consumption_users = filter_valid_users(DF_m2_Residential, Residential_Consumption, 'Chosen_Consumption')
    valid_industrial_consumption_users = filter_valid_users(DF_m2_Industrial, Industrial_Consumption, 'Chosen_Consumption')
    valid_commercial_consumption_users = filter_valid_users(DF_m2_Commercial, Commercial_Consumption, 'Chosen_Consumption')

    # Calculate production for each user type using the 'Chosen_Production' values
    week_production_residential = [
        sum(
            DF_m2_Residential.loc[user, 'Chosen_Production'] *
            Residential_user_production_df.loc[date, user] *
            DF_m2_Residential.loc[user, 'Area']
            for user in valid_residential_production_users if user in Residential_user_production_df.columns
        ) for date in week_data_index
    ]
    
    print(f"Sample Residential Production (Month {month}): {week_production_residential[:5]}...")  # Debug

    week_production_industrial = [
        sum(
            DF_m2_Industrial.loc[user, 'Chosen_Production'] *
            Industrial_user_production_df.loc[date, user] *
            DF_m2_Industrial.loc[user, 'Area']
            for user in valid_industrial_production_users if user in Industrial_user_production_df.columns
        ) for date in week_data_index
    ]

    print(f"Sample Industrial Production (Month {month}): {week_production_industrial[:5]}...")  # Debug

    week_production_commercial = [
        sum(
            DF_m2_Commercial.loc[user, 'Chosen_Production'] *
            Commercial_user_production_df.loc[date, user] *
            DF_m2_Commercial.loc[user, 'Area']
            for user in valid_commercial_production_users if user in Commercial_user_production_df.columns
        ) for date in week_data_index
    ]

    print(f"Sample Commercial Production (Month {month}): {week_production_commercial[:5]}...")  # Debug

    # Calculate total production for each hour
    week_production_total = [
        week_production_residential[i] + week_production_industrial[i] + week_production_commercial[i]
        for i in range(len(week_data_index))
    ]

    print(f"Sample Total Production (Month {month}): {week_production_total[:5]}...")  # Debug

    # Calculate total consumption for each user category using 'Chosen_Consumption' values
    week_consumption_residential = [
        sum(
            DF_m2_Residential.loc[user, 'Chosen_Consumption'] *
            Residential_Consumption.loc[date, user] *
            DF_m2_Residential.loc[user, 'Area']
            for user in valid_residential_consumption_users if user in Residential_Consumption.columns
        ) for date in week_data_index
    ]
    print(f"Sample Residential Consumption (Month {month}): {week_consumption_residential[:5]}...")  # Debug

    week_consumption_industrial = [
        sum(
            DF_m2_Industrial.loc[user, 'Chosen_Consumption'] *
            Industrial_Consumption.loc[date, user] *
            DF_m2_Industrial.loc[user, 'Area']
            for user in valid_industrial_consumption_users if user in Industrial_Consumption.columns
        ) for date in week_data_index
    ]
    print(f"Sample Industrial Consumption (Month {month}): {week_consumption_industrial[:5]}...")  # Debug

    week_consumption_commercial = [
        sum(
            DF_m2_Commercial.loc[user, 'Chosen_Consumption'] *
            Commercial_Consumption.loc[date, user] *
            DF_m2_Commercial.loc[user, 'Area']
            for user in valid_commercial_consumption_users if user in Commercial_Consumption.columns
        ) for date in week_data_index
    ]
    print(f"Sample Commercial Consumption (Month {month}): {week_consumption_commercial[:5]}...")  # Debug

    # Calculate total consumption for each hour
    week_consumption_total = [
        week_consumption_residential[i] + week_consumption_industrial[i] + week_consumption_commercial[i]
        for i in range(len(week_data_index))
    ]

    print(f"Sample Total Consumption (Month {month}): {week_consumption_total[:5]}...")  # Debug

    # Define F1, F2, F3 periods and 10am to 4pm
    F1_mask = (week_data_index.weekday < 5) & (week_data_index.hour >= 8) & (week_data_index.hour < 19)
    F2_mask = ((week_data_index.weekday < 5) & ((week_data_index.hour >= 7) & (week_data_index.hour < 8) | (week_data_index.hour >= 19) & (week_data_index.hour < 23))) | \
              ((week_data_index.weekday == 5) & (week_data_index.hour >= 7) & (week_data_index.hour < 23))
    F3_mask = ((week_data_index.weekday < 6) & (week_data_index.hour >= 23)) | (week_data_index.hour < 7) | (week_data_index.weekday == 6)
    time_10am_4pm_mask = (week_data_index.hour >= 10) & (week_data_index.hour < 16)

    # Calculate total production and consumption for each period
    total_production_F1 = sum([week_production_total[i] for i in range(len(week_data_index)) if F1_mask[i]])
    total_consumption_F1 = sum([week_consumption_total[i] for i in range(len(week_data_index)) if F1_mask[i]])
    F1_percentage = (total_production_F1 / total_consumption_F1) * 100 if total_consumption_F1 > 0 else 0

    total_production_F2 = sum([week_production_total[i] for i in range(len(week_data_index)) if F2_mask[i]])
    total_consumption_F2 = sum([week_consumption_total[i] for i in range(len(week_data_index)) if F2_mask[i]])
    F2_percentage = (total_production_F2 / total_consumption_F2) * 100 if total_consumption_F2 > 0 else 0

    total_production_F3 = sum([week_production_total[i] for i in range(len(week_data_index)) if F3_mask[i]])
    total_consumption_F3 = sum([week_consumption_total[i] for i in range(len(week_data_index)) if F3_mask[i]])
    F3_percentage = (total_production_F3 / total_consumption_F3) * 100 if total_consumption_F3 > 0 else 0

    total_production_10am_4pm = sum([week_production_total[i] for i in range(len(week_data_index)) if time_10am_4pm_mask[i]])
    total_consumption_10am_4pm = sum([week_consumption_total[i] for i in range(len(week_data_index)) if time_10am_4pm_mask[i]])
    percentage_10am_4pm = (total_production_10am_4pm / total_consumption_10am_4pm) * 100 if total_consumption_10am_4pm > 0 else 0

    # Append results for each month
    weekly_results['Month'].append(month)
    weekly_results['F1_Percentage'].append(F1_percentage)
    weekly_results['F2_Percentage'].append(F2_percentage)
    weekly_results['F3_Percentage'].append(F3_percentage)
    weekly_results['10am_to_4pm_Percentage'].append(percentage_10am_4pm)

    # Plotting function
    def plot_production_vs_consumption(time_series, production, consumption, label, color, month):
        plt.figure(figsize=(12, 6))
        plt.plot(time_series, production, label=f'{label} Production', color=color)
        plt.plot(time_series, consumption, label='Total Consumption', color='red')
        plt.xlabel('Time')
        plt.ylabel('Energy (W)')
        plt.title(f'{label} Production vs. Total Consumption (First Week of Month {month:02d})')
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(weekly_plot_dir, f'{label}_Production_vs_Total_Consumption_Week_{month:02d}.png'))
        plt.show()

    # Plot total production vs total consumption
    plot_production_vs_consumption(week_data_index, week_production_total, week_consumption_total, 'Total', 'blue', month)

    # Plot residential production vs total consumption
    plot_production_vs_consumption(week_data_index, week_production_residential, week_consumption_total, 'Residential', 'green', month)

    # Plot industrial production vs total consumption
    plot_production_vs_consumption(week_data_index, week_production_industrial, week_consumption_total, 'Industrial', 'purple', month)

    # Plot commercial production vs total consumption
    plot_production_vs_consumption(week_data_index, week_production_commercial, week_consumption_total, 'Commercial', 'orange', month)

# Convert results to DataFrame
weekly_results_df = pd.DataFrame(weekly_results)

# Save the results to Excel and CSV
output_file_path_excel = os.path.join(weekly_plot_dir, 'Weekly_Percentage_Results.xlsx')
output_file_path_csv = os.path.join(weekly_plot_dir, 'Weekly_Percentage_Results.csv')

weekly_results_df.to_excel(output_file_path_excel, index=False)
weekly_results_df.to_csv(output_file_path_csv, index=False)

print(f'Results saved to {output_file_path_excel} and {output_file_path_csv}')



In [ ]:
import matplotlib.pyplot as plt

# Define July 1st date range
july_1st_date = '2019-07-01'
july_1st_data_index = pd.date_range(start=f'{july_1st_date} 00:00:00', end=f'{july_1st_date} 23:00:00', freq='H')

# Calculate total production for July 1st
july_1st_total_production = [
    sum(
        DF_m2_Residential.loc[user, 'Chosen_Production'] *
        Residential_user_production_df.loc[date, user] *
        DF_m2_Residential.loc[user, 'Area']
        for user in valid_residential_production_users if user in Residential_user_production_df.columns
    ) +
    sum(
        DF_m2_Industrial.loc[user, 'Chosen_Production'] *
        Industrial_user_production_df.loc[date, user] *
        DF_m2_Industrial.loc[user, 'Area']
        for user in valid_industrial_production_users if user in Industrial_user_production_df.columns
    ) +
    sum(
        DF_m2_Commercial.loc[user, 'Chosen_Production'] *
        Commercial_user_production_df.loc[date, user] *
        DF_m2_Commercial.loc[user, 'Area']
        for user in valid_commercial_production_users if user in Commercial_user_production_df.columns
    )
    for date in july_1st_data_index
]

# Calculate total consumption for July 1st
july_1st_total_consumption = [
    sum(
        DF_m2_Residential.loc[user, 'Chosen_Consumption'] *
        Residential_Consumption.loc[date, user] *
        DF_m2_Residential.loc[user, 'Area']
        for user in valid_residential_consumption_users if user in Residential_Consumption.columns
    ) +
    sum(
        DF_m2_Industrial.loc[user, 'Chosen_Consumption'] *
        Industrial_Consumption.loc[date, user] *
        DF_m2_Industrial.loc[user, 'Area']
        for user in valid_industrial_consumption_users if user in Industrial_Consumption.columns
    ) +
    sum(
        DF_m2_Commercial.loc[user, 'Chosen_Consumption'] *
        Commercial_Consumption.loc[date, user] *
        DF_m2_Commercial.loc[user, 'Area']
        for user in valid_commercial_consumption_users if user in Commercial_Consumption.columns
    )
    for date in july_1st_data_index
]

# Attempt to plot the total production and total consumption for July 1st
plt.figure(figsize=(12, 6))
plt.plot(july_1st_data_index, july_1st_total_production, label='Total Production', color='blue')
plt.plot(july_1st_data_index, july_1st_total_consumption, label='Total Consumption', color='red')
plt.xlabel('Time')
plt.ylabel('Energy (W)')
plt.title('Total Production vs Total Consumption on July 1st')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
#################vecchio codice ################################ààà


import os
import pandas as pd
import matplotlib.pyplot as plt

# Ensure all production and consumption data indices are in datetime format
Residential_user_production_df.index = pd.to_datetime(Residential_user_production_df.index)
Industrial_user_production_df.index = pd.to_datetime(Industrial_user_production_df.index)
Commercial_user_production_df.index = pd.to_datetime(Commercial_user_production_df.index)

Residential_Consumption.index = pd.to_datetime(Residential_Consumption.index)
Industrial_Consumption.index = pd.to_datetime(Industrial_Consumption.index)
Commercial_Consumption.index = pd.to_datetime(Commercial_Consumption.index)

# Set 'full_id' as the index for the DF_m2_* DataFrames
DF_m2_Residential.set_index('full_id', inplace=True)
DF_m2_Industrial.set_index('full_id', inplace=True)
DF_m2_Commercial.set_index('full_id', inplace=True)

# Strip any extra spaces in 'full_id' and production DataFrame column names
DF_m2_Residential.index = DF_m2_Residential.index.astype(str).str.strip()
DF_m2_Industrial.index = DF_m2_Industrial.index.astype(str).str.strip()
DF_m2_Commercial.index = DF_m2_Commercial.index.astype(str).str.strip()

Residential_user_production_df.columns = Residential_user_production_df.columns.astype(str).str.strip()
Industrial_user_production_df.columns = Industrial_user_production_df.columns.astype(str).str.strip()
Commercial_user_production_df.columns = Commercial_user_production_df.columns.astype(str).str.strip()

# Initialize directories for saving plots
plot_dir = os.path.join(results_dir, 'Plot')
weekly_plot_dir = os.path.join(plot_dir, 'Weekly_Plot')
os.makedirs(weekly_plot_dir, exist_ok=True)

# Initialize a dictionary to store the results
weekly_results = {
    'Month': [],
    'F1_Percentage': [],
    'F2_Percentage': [],
    'F3_Percentage': [],
    '10am_to_4pm_Percentage': []
}

# Print a sample of identifiers for debugging
print(f"Sample Residential full_id in DF_m2_Residential: {DF_m2_Residential.index[:5].tolist()}")
print(f"Sample columns in Residential_user_production_df: {Residential_user_production_df.columns[:5].tolist()}")

# Function to filter users that are present in the DataFrame columns
def filter_valid_users(df, production_df):
    valid_users = [user for user in df.index if user in production_df.columns]
    print(f"Valid users: {valid_users[:5]}...")  # Debug: print first 5 valid users
    return valid_users

# Loop through each month to calculate percentages for each period and plot the first week of each month
for month in range(1, 13):
    start_date = f'2019-{month:02d}-01'
    end_date = f'2019-{month:02d}-07'

    # Filter data for the first week of the month
    week_data_index = pd.date_range(start=start_date, end=end_date, freq='H')

    # Filter valid users based on their presence in production data
    valid_residential_users = filter_valid_users(DF_m2_Residential, Residential_user_production_df)
    valid_industrial_users = filter_valid_users(DF_m2_Industrial, Industrial_user_production_df)
    valid_commercial_users = filter_valid_users(DF_m2_Commercial, Commercial_user_production_df)

    # Calculate production for each user type using the 'Chosen_Production' values
    week_production_residential = [
        sum(
            DF_m2_Residential.loc[user, 'Chosen_Production'] *
            Residential_user_production_df.loc[date, user] *
            DF_m2_Residential.loc[user, 'Area']
            for user in valid_residential_users if user in Residential_user_production_df.columns
        ) for date in week_data_index
    ]
    
    print(f"Sample Residential Production (Month {month}): {week_production_residential[:5]}...")  # Debug

    week_production_industrial = [
        sum(
            DF_m2_Industrial.loc[user, 'Chosen_Production'] *
            Industrial_user_production_df.loc[date, user] *
            DF_m2_Industrial.loc[user, 'Area']
            for user in valid_industrial_users if user in Industrial_user_production_df.columns
        ) for date in week_data_index
    ]

    print(f"Sample Industrial Production (Month {month}): {week_production_industrial[:5]}...")  # Debug

    week_production_commercial = [
        sum(
            DF_m2_Commercial.loc[user, 'Chosen_Production'] *
            Commercial_user_production_df.loc[date, user] *
            DF_m2_Commercial.loc[user, 'Area']
            for user in valid_commercial_users if user in Commercial_user_production_df.columns
        ) for date in week_data_index
    ]

    print(f"Sample Commercial Production (Month {month}): {week_production_commercial[:5]}...")  # Debug

    # Calculate total production for each hour
    week_production_total = [
        week_production_residential[i] + week_production_industrial[i] + week_production_commercial[i]
        for i in range(len(week_data_index))
    ]

    print(f"Sample Total Production (Month {month}): {week_production_total[:5]}...")  # Debug

    # Calculate total consumption for the week for each user category using 'Chosen_Consumption' values
    week_consumption_residential = [
        sum(
            DF_m2_Residential.loc[user, 'Chosen_Consumption'] *
            Residential_Consumption.loc[date, user] *
            DF_m2_Residential.loc[user, 'Area']
            for user in valid_residential_users if user in Residential_Consumption.columns
        ) for date in week_data_index
    ]
    print(f"Sample Residential Consumption (Month {month}): {week_consumption_residential[:5]}...")  # Debug

    week_consumption_industrial = [
        sum(
            DF_m2_Industrial.loc[user, 'Chosen_Consumption'] *
            Industrial_Consumption.loc[date, user] *
            DF_m2_Industrial.loc[user, 'Area']
            for user in valid_industrial_users if user in Industrial_Consumption.columns
        ) for date in week_data_index
    ]
    print(f"Sample Industrial Consumption (Month {month}): {week_consumption_industrial[:5]}...")  # Debug

    week_consumption_commercial = [
        sum(
            DF_m2_Commercial.loc[user, 'Chosen_Consumption'] *
            Commercial_Consumption.loc[date, user] *
            DF_m2_Commercial.loc[user, 'Area']
            for user in valid_commercial_users if user in Commercial_Consumption.columns
        ) for date in week_data_index
    ]
    print(f"Sample Commercial Consumption (Month {month}): {week_consumption_commercial[:5]}...")  # Debug

    # Calculate total consumption for each hour
    week_consumption_total = [
        week_consumption_residential[i] + week_consumption_industrial[i] + week_consumption_commercial[i]
        for i in range(len(week_data_index))
    ]

    print(f"Sample Total Consumption (Month {month}): {week_consumption_total[:5]}...")  # Debug


    # Define F1, F2, F3 periods and 10am to 4pm
    F1_mask = (week_data_index.weekday < 5) & (week_data_index.hour >= 8) & (week_data_index.hour < 19)
    F2_mask = ((week_data_index.weekday < 5) & ((week_data_index.hour >= 7) & (week_data_index.hour < 8) | (week_data_index.hour >= 19) & (week_data_index.hour < 23))) | \
              ((week_data_index.weekday == 5) & (week_data_index.hour >= 7) & (week_data_index.hour < 23))
    F3_mask = ((week_data_index.weekday < 6) & (week_data_index.hour >= 23)) | (week_data_index.hour < 7) | (week_data_index.weekday == 6)
    time_10am_4pm_mask = (week_data_index.hour >= 10) & (week_data_index.hour < 16)

    # Calculate total production and consumption for each period
    total_production_F1 = sum([week_production_total[i] for i in range(len(week_data_index)) if F1_mask[i]])
    total_consumption_F1 = sum([week_consumption_total[i] for i in range(len(week_data_index)) if F1_mask[i]])
    F1_percentage = (total_production_F1 / total_consumption_F1) * 100 if total_consumption_F1 > 0 else 0

    total_production_F2 = sum([week_production_total[i] for i in range(len(week_data_index)) if F2_mask[i]])
    total_consumption_F2 = sum([week_consumption_total[i] for i in range(len(week_data_index)) if F2_mask[i]])
    F2_percentage = (total_production_F2 / total_consumption_F2) * 100 if total_consumption_F2 > 0 else 0

    total_production_F3 = sum([week_production_total[i] for i in range(len(week_data_index)) if F3_mask[i]])
    total_consumption_F3 = sum([week_consumption_total[i] for i in range(len(week_data_index)) if F3_mask[i]])
    F3_percentage = (total_production_F3 / total_consumption_F3) * 100 if total_consumption_F3 > 0 else 0

    total_production_10am_4pm = sum([week_production_total[i] for i in range(len(week_data_index)) if time_10am_4pm_mask[i]])
    total_consumption_10am_4pm = sum([week_consumption_total[i] for i in range(len(week_data_index)) if time_10am_4pm_mask[i]])
    percentage_10am_4pm = (total_production_10am_4pm / total_consumption_10am_4pm) * 100 if total_consumption_10am_4pm > 0 else 0

    # Append results for each month
    weekly_results['Month'].append(month)
    weekly_results['F1_Percentage'].append(F1_percentage)
    weekly_results['F2_Percentage'].append(F2_percentage)
    weekly_results['F3_Percentage'].append(F3_percentage)
    weekly_results['10am_to_4pm_Percentage'].append(percentage_10am_4pm)

    # Plotting function
    def plot_production_vs_consumption(time_series, production, consumption, label, color, month):
        plt.figure(figsize=(12, 6))
        plt.plot(time_series, production, label=f'{label} Production', color=color)
        plt.plot(time_series, consumption, label='Total Consumption', color='red')
        plt.xlabel('Time')
        plt.ylabel('Energy (W)')
        plt.title(f'{label} Production vs. Total Consumption (First Week of Month {month:02d})')
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(weekly_plot_dir, f'{label}_Production_vs_Total_Consumption_Week_{month:02d}.png'))
        plt.show()

    # Plot total production vs total consumption
    plot_production_vs_consumption(week_data_index, week_production_total, week_consumption_total, 'Total', 'blue', month)

    # Plot residential production vs total consumption
    plot_production_vs_consumption(week_data_index, week_production_residential, week_consumption_total, 'Residential', 'green', month)

    # Plot industrial production vs total consumption
    plot_production_vs_consumption(week_data_index, week_production_industrial, week_consumption_total, 'Industrial', 'purple', month)

    # Plot commercial production vs total consumption
    plot_production_vs_consumption(week_data_index, week_production_commercial, week_consumption_total, 'Commercial', 'orange', month)

# Convert results to DataFrame
weekly_results_df = pd.DataFrame(weekly_results)

# Save the results to Excel and CSV
output_file_path_excel = os.path.join(weekly_plot_dir, 'Weekly_Percentage_Results.xlsx')
output_file_path_csv = os.path.join(weekly_plot_dir, 'Weekly_Percentage_Results.csv')

weekly_results_df.to_excel(output_file_path_excel, index=False)
weekly_results_df.to_csv(output_file_path_csv, index=False)

print(f'Results saved to {output_file_path_excel} and {output_file_path_csv}')

## Adding Dynamic Penalization

In [ ]:
import pandas as pd
import os
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from datetime import datetime

# Paths to directories
Data_path_GDrive = "G:/Il mio Drive/Thesis_Large_Files/Working Table"
Data_path_Production = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Production"
Data_path_Consumption = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption"
Data_path_Users = "C:/Users/Nik/Documents/GitHub/Thesis/CSV/Users"
results_dir = 'C:/Users/Nik/Documents/GitHub/Thesis/Results/Results_Binary/Final Results'
results_dir = os.path.join(results_dir, datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + '_BinaryProductionandConsumption')
os.makedirs(results_dir, exist_ok=True)

# Load Working Tables for every optimization try
WorkingTables_Path = 'C:/Users/Nik/Documents/GitHub/Thesis/CSV/Working Tables'
DF_m2_Residential = pd.read_csv(os.path.join(WorkingTables_Path, 'DF_m2_Residential.csv'), sep=",", on_bad_lines='skip', header=0)
DF_m2_Industrial = pd.read_csv(os.path.join(WorkingTables_Path, 'DF_m2_Industrial.csv'), sep=",", on_bad_lines='skip', header=0)
DF_m2_Commercial = pd.read_csv(os.path.join(WorkingTables_Path, 'DF_m2_Commercial.csv'), sep=",", on_bad_lines='skip', header=0)

Residential_user_production_df = pd.read_csv(os.path.join(Data_path_GDrive, 'Residential_user_production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])
Industrial_user_production_df = pd.read_csv(os.path.join(WorkingTables_Path, 'Industrial_user_production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])
Commercial_user_production_df = pd.read_csv(os.path.join(WorkingTables_Path, 'Commercial_user_production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])

# Load New Consumption Curves
Residential_Consumption = pd.read_csv(os.path.join(Data_path_GDrive, 'Residential_Consumption_Optimized_From_Switched_2.csv'), index_col=0, parse_dates=True)
Industrial_Consumption = pd.read_csv('C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption/Industrial_Consumption_Optimized_From_Switched_2.csv', index_col=0, parse_dates=True)
Commercial_Consumption = pd.read_csv('C:/Users/Nik/Documents/GitHub/Thesis/CSV/Consumption/Commercial_Consumption_Optimized_From_Switched_2.csv', index_col=0, parse_dates=True)

DF_Production = pd.read_csv(os.path.join(Data_path_Production, '2019_Production.csv'), sep=",", on_bad_lines='skip', index_col=0, header=0, parse_dates=[0])

# Use 'full_id' as the primary identifier
residential_users = DF_m2_Residential['full_id'].astype(str).tolist()
industrial_users = DF_m2_Industrial['full_id'].astype(str).tolist()
commercial_users = DF_m2_Commercial['full_id'].astype(str).tolist()

# Precompute area mappings to avoid repeated DataFrame lookups
residential_area_dict = DF_m2_Residential.set_index('full_id')['Area'].to_dict()
industrial_area_dict = DF_m2_Industrial.set_index('full_id')['Area'].to_dict()
commercial_area_dict = DF_m2_Commercial.set_index('full_id')['Area'].to_dict()

# Define the yearlist based on the number of hours
yearlist = DF_Production.index

# Fixed consumption area to achieve
Fixed_Res_Consumption_Area = 50000  # Example value
Fixed_Ind_Consumption_Area = 1000   # Example value
Fixed_Com_Consumption_Area = 2000   # Example value

# Create a new Gurobi model
m = gp.Model("Optimization_Model_ConsumptionProduction")

# Production and Consumption Binary Variables [0;1]
binary_vars_production_residential = m.addVars(residential_users, vtype=GRB.BINARY, name="binary_production_residential")
binary_vars_consumption_residential = m.addVars(residential_users, vtype=GRB.BINARY, name="binary_consumption_residential")

binary_vars_production_industrial = m.addVars(industrial_users, vtype=GRB.BINARY, name="binary_production_industrial")
binary_vars_consumption_industrial = m.addVars(industrial_users, vtype=GRB.BINARY, name="binary_consumption_industrial")

binary_vars_production_commercial = m.addVars(commercial_users, vtype=GRB.BINARY, name="binary_production_commercial")
binary_vars_consumption_commercial = m.addVars(commercial_users, vtype=GRB.BINARY, name="binary_consumption_commercial")

# Auxiliary variable for absolute deviation
deviation = m.addVars(yearlist, lb=0, vtype=GRB.CONTINUOUS, name="deviation")

# Initial Penalty Coefficients
penalty_coefficient_production = 50  # Initial penalty for not selecting production users
penalty_coefficient_consumption = 200  # Initial penalty for not selecting consumption users

# Maximum number of iterations for dynamic adjustment
max_iterations = 10
tolerance = 0.05  # Tolerance for balance between production and consumption

# Pre-compute consumption and production terms for each user and hour
residential_prod_terms = {
    t: {user: Residential_user_production_df.loc[t, user] for user in residential_users if user in Residential_user_production_df.columns}
    for t in yearlist
}
industrial_prod_terms = {
    t: {user: Industrial_user_production_df.loc[t, user] for user in industrial_users if user in Industrial_user_production_df.columns}
    for t in yearlist
}
commercial_prod_terms = {
    t: {user: Commercial_user_production_df.loc[t, user] for user in commercial_users if user in Commercial_user_production_df.columns}
    for t in yearlist
}

residential_cons_terms = {
    t: {user: Residential_Consumption[f'Residential_User_{user}_Mixed'].loc[t] * residential_area_dict[user]
        for user in residential_users if f'Residential_User_{user}_Mixed' in Residential_Consumption.columns}
    for t in yearlist
}
industrial_cons_terms = {
    t: {user: Industrial_Consumption[f'Industrial_User_{user}_Mixed'].loc[t] * industrial_area_dict[user]
        for user in industrial_users if f'Industrial_User_{user}_Mixed' in Industrial_Consumption.columns}
    for t in yearlist
}
commercial_cons_terms = {
    t: {user: Commercial_Consumption[f'Commercial_User_{user}_Mixed'].loc[t] * commercial_area_dict[user]
        for user in commercial_users if f'Commercial_User_{user}_Mixed' in Commercial_Consumption.columns}
    for t in yearlist
}

print("Finish precomputing")

# Calculate the difference between production and consumption for each hour using precomputed terms
for t in yearlist:
    production_res = gp.quicksum(binary_vars_production_residential[user] * residential_prod_terms[t].get(user, 0) for user in residential_users)
    consumption_res = gp.quicksum(binary_vars_consumption_residential[user] * residential_cons_terms[t].get(user, 0) for user in residential_users)
    
    production_ind = gp.quicksum(binary_vars_production_industrial[user] * industrial_prod_terms[t].get(user, 0) for user in industrial_users)
    consumption_ind = gp.quicksum(binary_vars_consumption_industrial[user] * industrial_cons_terms[t].get(user, 0) for user in industrial_users)
    
    production_com = gp.quicksum(binary_vars_production_commercial[user] * commercial_prod_terms[t].get(user, 0) for user in commercial_users)
    consumption_com = gp.quicksum(binary_vars_consumption_commercial[user] * commercial_cons_terms[t].get(user, 0) for user in commercial_users)
    
    difference = production_res - consumption_res + production_ind - consumption_ind + production_com - consumption_com
    
    # Add constraints for deviation
    m.addConstr(deviation[t] >= difference, name=f"PosDeviation_{t}")
    m.addConstr(deviation[t] >= -difference, name=f"NegDeviation_{t}")

# Constraint: Total selected consumption area must equal or exceed the fixed value
m.addConstr(
    gp.quicksum(binary_vars_consumption_residential[user] * residential_area_dict[user] for user in residential_users) >= Fixed_Res_Consumption_Area,
    name="FixedResConsumptionArea"
)

m.addConstr(
    gp.quicksum(binary_vars_consumption_industrial[user] * industrial_area_dict[user] for user in industrial_users) >= Fixed_Ind_Consumption_Area,
    name="FixedIndConsumptionArea"
)

m.addConstr(
    gp.quicksum(binary_vars_consumption_commercial[user] * commercial_area_dict[user] for user in commercial_users) >= Fixed_Com_Consumption_Area,
    name="FixedComConsumptionArea"
)

# Function to calculate imbalance
def calculate_imbalance(num_selected_production, num_selected_consumption, total_production, total_consumption):
    production_ratio = num_selected_production / total_production if total_production > 0 else 0
    consumption_ratio = num_selected_consumption / total_consumption if total_consumption > 0 else 0
    return production_ratio, consumption_ratio

# Iterative optimization with dynamic penalty adjustment
for iteration in range(max_iterations):
    print(f"Iteration {iteration + 1} with penalties: Production = {penalty_coefficient_production}, Consumption = {penalty_coefficient_consumption}")

    # Objective function: minimize the deviation and penalize unselected users with different penalties for production and consumption
    penalty = (
        penalty_coefficient_production * (
            gp.quicksum(1 - binary_vars_production_residential[user] for user in residential_users) +
            gp.quicksum(1 - binary_vars_production_industrial[user] for user in industrial_users) +
            gp.quicksum(1 - binary_vars_production_commercial[user] for user in commercial_users)
        ) +
        penalty_coefficient_consumption * (
            gp.quicksum(1 - binary_vars_consumption_residential[user] for user in residential_users) +
            gp.quicksum(1 - binary_vars_consumption_industrial[user] for user in industrial_users) +
            gp.quicksum(1 - binary_vars_consumption_commercial[user] for user in commercial_users)
        )
    )

    # Minimize deviation and add penalization
    m.setObjective(gp.quicksum(deviation[t] for t in yearlist) + penalty, GRB.MINIMIZE)

    # Optimize the model
    m.optimize()

    # Check for solution status
    if m.Status == GRB.INFEASIBLE:
        print("Model is infeasible.")
        m.computeIIS()  # Compute Irreducible Inconsistent Subsystem (IIS) to understand infeasibility
        m.write(os.path.join(results_dir, "model.ilp"))  # Save IIS to a file
        break
    elif m.Status in [GRB.UNBOUNDED, GRB.INF_OR_UNBD]:
        print("Model is unbounded or infeasible.")
        break
    elif m.Status == GRB.TIME_LIMIT:
        print("Optimization stopped due to time limit.")
    elif m.Status == GRB.OPTIMAL:
        print("Optimal solution found!")

    # Extract results for each user type
    chosen_residential_production = [binary_vars_production_residential[user].X for user in residential_users]
    chosen_residential_consumption = [binary_vars_consumption_residential[user].X for user in residential_users]
    chosen_industrial_production = [binary_vars_production_industrial[user].X for user in industrial_users]
    chosen_industrial_consumption = [binary_vars_consumption_industrial[user].X for user in industrial_users]
    chosen_commercial_production = [binary_vars_production_commercial[user].X for user in commercial_users]
    chosen_commercial_consumption = [binary_vars_consumption_commercial[user].X for user in commercial_users]

    # Calculate the number of chosen users
    num_residential_chosen_production = sum(chosen_residential_production)
    num_residential_chosen_consumption = sum(chosen_residential_consumption)
    num_industrial_chosen_production = sum(chosen_industrial_production)
    num_industrial_chosen_consumption = sum(chosen_industrial_consumption)
    num_commercial_chosen_production = sum(chosen_commercial_production)
    num_commercial_chosen_consumption = sum(chosen_commercial_consumption)

    total_production_users = len(residential_users) + len(industrial_users) + len(commercial_users)
    total_consumption_users = total_production_users  # Assuming each user can be both producer and consumer

    # Calculate imbalance between production and consumption
    production_ratio, consumption_ratio = calculate_imbalance(
        num_residential_chosen_production + num_industrial_chosen_production + num_commercial_chosen_production,
        num_residential_chosen_consumption + num_industrial_chosen_consumption + num_commercial_chosen_consumption,
        total_production_users,
        total_consumption_users
    )

    print(f"Production Ratio: {production_ratio:.2f}, Consumption Ratio: {consumption_ratio:.2f}")

    # Check if the ratios are within tolerance
    if abs(production_ratio - consumption_ratio) <= tolerance:
        print("Balance achieved within tolerance.")
        break

    # Adjust penalties based on imbalance
    if production_ratio > consumption_ratio:
        # More production users selected, increase penalty for production
        penalty_coefficient_production += 20
        penalty_coefficient_consumption -= 10
    else:
        # More consumption users selected, increase penalty for consumption
        penalty_coefficient_consumption += 20
        penalty_coefficient_production -= 10

    # Ensure penalties are not negative
    penalty_coefficient_production = max(10, penalty_coefficient_production)
    penalty_coefficient_consumption = max(10, penalty_coefficient_consumption)

print("Dynamic penalty adjustment complete.")

# Extract results for each user type and save to CSV files
RP_values = {user: binary_vars_production_residential[user].X for user in residential_users}
CP_values = {user: binary_vars_consumption_residential[user].X for user in residential_users}
IP_values = {user: binary_vars_production_industrial[user].X for user in industrial_users}
CIP_values = {user: binary_vars_consumption_industrial[user].X for user in industrial_users}
CP_values_com = {user: binary_vars_production_commercial[user].X for user in commercial_users}
CIP_values_com = {user: binary_vars_consumption_commercial[user].X for user in commercial_users}

# Create new DataFrames with the chosen binary variables
DF_m2_Residential['Chosen_Production'] = [RP_values[user] for user in DF_m2_Residential['full_id']]
DF_m2_Residential['Chosen_Consumption'] = [CP_values[user] for user in DF_m2_Residential['full_id']]
DF_m2_Industrial['Chosen_Production'] = [IP_values[user] for user in DF_m2_Industrial['full_id']]
DF_m2_Industrial['Chosen_Consumption'] = [CIP_values[user] for user in DF_m2_Industrial['full_id']]
DF_m2_Commercial['Chosen_Production'] = [CP_values_com[user] for user in DF_m2_Commercial['full_id']]
DF_m2_Commercial['Chosen_Consumption'] = [CIP_values_com[user] for user in DF_m2_Commercial['full_id']]

# Extract chosen users based on optimization results
chosen_residential_production = DF_m2_Residential[DF_m2_Residential['Chosen_Production'] == 1]
chosen_residential_consumption = DF_m2_Residential[DF_m2_Residential['Chosen_Consumption'] == 1]
chosen_industrial_production = DF_m2_Industrial[DF_m2_Industrial['Chosen_Production'] == 1]
chosen_industrial_consumption = DF_m2_Industrial[DF_m2_Industrial['Chosen_Consumption'] == 1]
chosen_commercial_production = DF_m2_Commercial[DF_m2_Commercial['Chosen_Production'] == 1]
chosen_commercial_consumption = DF_m2_Commercial[DF_m2_Commercial['Chosen_Consumption'] == 1]

# Save the results to CSV files
chosen_residential_production.to_csv(os.path.join(results_dir, 'chosen_residential_production.csv'))
chosen_residential_consumption.to_csv(os.path.join(results_dir, 'chosen_residential_consumption.csv'))
chosen_industrial_production.to_csv(os.path.join(results_dir, 'chosen_industrial_production.csv'))
chosen_industrial_consumption.to_csv(os.path.join(results_dir, 'chosen_industrial_consumption.csv'))
chosen_commercial_production.to_csv(os.path.join(results_dir, 'chosen_commercial_production.csv'))
chosen_commercial_consumption.to_csv(os.path.join(results_dir, 'chosen_commercial_consumption.csv'))

# Calculate the number of chosen users versus the total number of users for each category
num_residential_chosen_production = len(chosen_residential_production)
num_residential_total = len(DF_m2_Residential)

num_residential_chosen_consumption = len(chosen_residential_consumption)
num_industrial_chosen_production = len(chosen_industrial_production)
num_industrial_total = len(DF_m2_Industrial)

num_industrial_chosen_consumption = len(chosen_industrial_consumption)
num_commercial_chosen_production = len(chosen_commercial_production)
num_commercial_total = len(DF_m2_Commercial)

num_commercial_chosen_consumption = len(chosen_commercial_consumption)

# Print the amounts
print(f'Residential Production: Chosen = {num_residential_chosen_production}, Total = {num_residential_total}')
print(f'Residential Consumption: Chosen = {num_residential_chosen_consumption}, Total = {num_residential_total}')
print(f'Industrial Production: Chosen = {num_industrial_chosen_production}, Total = {num_industrial_total}')
print(f'Industrial Consumption: Chosen = {num_industrial_chosen_consumption}, Total = {num_industrial_total}')
print(f'Commercial Production: Chosen = {num_commercial_chosen_production}, Total = {num_commercial_total}')
print(f'Commercial Consumption: Chosen = {num_commercial_chosen_consumption}, Total = {num_commercial_total}')

# Final results and analysis
print(f'Objective Value: {m.objVal}')
print("Optimization and dynamic penalty adjustment process complete.")




# Plots

In [ ]:
import matplotlib.pyplot as plt

# Calculate total yearly chosen production and consumption
total_yearly_production = chosen_residential_production['Area'].sum() + chosen_industrial_production['Area'].sum() + chosen_commercial_production['Area'].sum()
total_yearly_consumption = chosen_residential_consumption['Area'].sum() + chosen_industrial_consumption['Area'].sum() + chosen_commercial_consumption['Area'].sum()

# Plot
plt.figure(figsize=(8, 6))
plt.bar(['Production', 'Consumption'], [total_yearly_production, total_yearly_consumption], color=['blue', 'orange'])
plt.title('Total Yearly Chosen Production vs Consumption')
plt.ylabel('Total Area (m²)')
plt.show()


In [ ]:
# Combine total consumption from all sources
total_consumption = Residential_Consumption + Industrial_Consumption + Commercial_Consumption

# Resample to weekly and extract the first week of each month
first_week_production = DF_Production.resample('W').sum()
first_week_consumption = total_consumption.resample('W').sum()

# Keep only the first week of each month
first_week_production = first_week_production.groupby(first_week_production.index.to_period("M")).head(1)
first_week_consumption = first_week_consumption.groupby(first_week_consumption.index.to_period("M")).head(1)

# Plot the result
plt.figure(figsize=(10, 6))
plt.plot(first_week_production.index, first_week_production.sum(axis=1), label='Production')
plt.plot(first_week_consumption.index, first_week_consumption.sum(axis=1), label='Consumption', color='orange')
plt.title('First Week of Each Month: Total Production and Consumption')
plt.ylabel('Total (kWh)')
plt.legend()
plt.show()


In [ ]:
import seaborn as sns

# Calculate the deficit/surplus (production - consumption)
deficit_surplus = first_week_production - first_week_total_consumption

# Create a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(deficit_surplus.T, cmap='coolwarm', cbar_kws={'label': 'Deficit (-) / Surplus (+)'})
plt.title('Heatmap of Deficit and Surplus (First Week of Each Month)')
plt.ylabel('Days')
plt.xlabel('Hours')
plt.show()


In [ ]:
# Resample data to monthly totals
monthly_production = DF_Production.resample('M').sum()
monthly_consumption = (Residential_Consumption + Industrial_Consumption + Commercial_Consumption).resample('M').sum()

# Plot
plt.figure(figsize=(10, 6))
plt.plot(monthly_production.index, monthly_production.sum(axis=1), label='Production')
plt.plot(monthly_consumption.index, monthly_consumption.sum(axis=1), label='Consumption', color='orange')
plt.title('Monthly Total Production and Consumption Variability')
plt.ylabel('Total (kWh)')
plt.legend()
plt.show()


In [ ]:
# Define seasons based on months
seasons = {
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Fall', 10: 'Fall', 11: 'Fall'
}

# Map the seasons
DF_Production['season'] = DF_Production.index.month.map(seasons)
seasonal_production = DF_Production.groupby('season').sum()
seasonal_consumption = (Residential_Consumption + Industrial_Consumption + Commercial_Consumption).groupby(Residential_Consumption.index.month.map(seasons)).sum()

# Plot
plt.figure(figsize=(10, 6))
plt.bar(seasonal_production.index, seasonal_production.sum(axis=1), label='Production')
plt.bar(seasonal_consumption.index, seasonal_consumption.sum(axis=1), label='Consumption', alpha=0.7, color='orange')
plt.title('Seasonal Total Production and Consumption Variability')
plt.ylabel('Total (kWh)')
plt.legend()
plt.show()


In [ ]:
# Filter for weekdays (Monday to Friday)
weekday_mask = DF_Production.index.weekday < 5

# Group by hour and calculate the mean for each hour during weekdays
weekday_production = DF_Production[weekday_mask].groupby(DF_Production[weekday_mask].index.hour).mean()
weekday_consumption = total_consumption[weekday_mask].groupby(total_consumption[weekday_mask].index.hour).mean()

# Plot
plt.figure(figsize=(10, 6))
plt.plot(weekday_production.index, weekday_production.sum(axis=1), label='Production')
plt.plot(weekday_consumption.index, weekday_consumption.sum(axis=1), label='Consumption', color='orange')
plt.title('Average Hourly Production and Consumption During Weekdays')
plt.ylabel('Average (kWh)')
plt.legend()
plt.show()


In [ ]:
# Filter for weekends
weekend_mask = DF_Production.index.weekday >= 5
weekend_production = DF_Production[weekend_mask].groupby(DF_Production.index.hour).mean()
weekend_consumption = (Residential_Consumption[weekend_mask] + Industrial_Consumption[weekend_mask] + Commercial_Consumption[weekend_mask]).groupby(Residential_Consumption.index.hour).mean()

# Plot
plt.figure(figsize=(10, 6))
plt.plot(weekend_production.index, weekend_production.sum(axis=1), label='Production')
plt.plot(weekend_consumption.index, weekend_consumption.sum(axis=1), label='Consumption', color='orange')
plt.title('Average Hourly Production and Consumption During Weekends')
plt.ylabel('Average (kWh)')
plt.legend()
plt.show()
